In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def calculate_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

def calculate_received_power(tx_power, distance, path_loss_exp, shadowing):
    path_loss = 10 * path_loss_exp * np.log10(distance)
    received_power = tx_power - path_loss + shadowing
    return received_power

def extract_enhanced_features(received_powers):
    """
    Extract basic statistical features from received power measurements
    to distinguish between PU and PUEA signals.

    Parameters:
        received_powers: Array of received power measurements

    Returns:
        List of statistical features
    """
    # Basic statistics (using only these five as requested)
    mean = np.mean(received_powers)
    variance = np.var(received_powers)
    median = np.median(received_powers)
    lower_quartile = np.percentile(received_powers, 25)
    upper_quartile = np.percentile(received_powers, 75)

    return [mean, variance, median, lower_quartile, upper_quartile]


def run_simulation(primary_power, puea_power, num_iterations, primary_position, puea_position, scenario_name):
    # Network parameters
    num_secondary_users = 20
    area_size = 100

    # Initialize separate result matrices for PU and PUEA for each time slot
    pu_result_matrix = []
    puea_result_matrix = []

    # Initialize arrays to store statistical features for each iteration
    pu_features = []
    puea_features = []

    # Calculate distance between PU and PUEA for reference
    pu_puea_distance = calculate_distance(primary_position, puea_position)

    # Plot network topology for this scenario
    plt.figure(figsize=(10, 10))
    plt.title(f'Scenario {scenario_name}: Network Topology\nPU-PUEA Distance: {pu_puea_distance:.2f} units')
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')

    # Generate positions of secondary users (constant for all iterations in this scenario)
    # Use ASCII value of first character as a seed offset
    seed_value = 42 + (ord(scenario_name[0]) - ord('A'))
    np.random.seed(seed_value)
    secondary_positions = np.random.uniform(0, area_size, size=(num_secondary_users, 2))

    # Plot secondary users
    plt.scatter(secondary_positions[:, 0], secondary_positions[:, 1],
               label='Secondary Users', c='blue', marker='o')

    # Plot primary user and PUEA
    plt.scatter(primary_position[0], primary_position[1],
               label='Primary User', c='red', marker='^', s=200)
    plt.scatter(puea_position[0], puea_position[1],
               label='PUEA', c='green', marker='s', s=200)

    # Annotate secondary users
    for i in range(num_secondary_users):
        plt.annotate(f'SU{i+1}', (secondary_positions[i, 0], secondary_positions[i, 1]))

    plt.grid(True)
    plt.legend()
    plt.axis([0, area_size, 0, area_size])
    #plt.savefig(f'scenario_{scenario_name}_topology.png')
    plt.show()

    # Time slot simulation
    print(f"\n===== Scenario {scenario_name} - Simulation over {num_iterations} Time Slots =====")
    print(f"PU Position: ({primary_position[0]}, {primary_position[1]})")
    print(f"PUEA Position: ({puea_position[0]}, {puea_position[1]})")
    print(f"PU-PUEA Distance: {pu_puea_distance:.2f} units")

    # Create arrays to store received powers for each time slot for visualization
    time_slots = np.arange(1, num_iterations + 1)
    avg_pu_power_per_slot = np.zeros(num_iterations)
    avg_puea_power_per_slot = np.zeros(num_iterations)

    for time_slot in range(num_iterations):
        # Add some randomness per time slot (represents changing channel conditions)
        np.random.seed(100 + time_slot)

        # Initialize arrays
        distances_to_primary = np.zeros(num_secondary_users)
        distances_to_puea = np.zeros(num_secondary_users)
        path_loss_exponents = np.random.uniform(2, 6, num_secondary_users)  # Range 2-6
        shadowing_values = np.random.uniform(4, 12, num_secondary_users)  # Range 4-12
        received_powers_primary = np.zeros(num_secondary_users)
        received_powers_puea = np.zeros(num_secondary_users)

        # Calculate features
        for i in range(num_secondary_users):
            distances_to_primary[i] = calculate_distance(secondary_positions[i], primary_position)
            distances_to_puea[i] = calculate_distance(secondary_positions[i], puea_position)

            received_powers_primary[i] = calculate_received_power(
                primary_power,
                distances_to_primary[i],
                path_loss_exponents[i],
                shadowing_values[i]
            )

            received_powers_puea[i] = calculate_received_power(
                puea_power,
                distances_to_puea[i],
                path_loss_exponents[i],
                shadowing_values[i]
            )
            print(f"PU and PUEA Pr :{received_powers_primary[i]},    {received_powers_puea[i]}")

        # Store average powers for this time slot
        avg_pu_power_per_slot[time_slot] = np.mean(received_powers_primary)
        avg_puea_power_per_slot[time_slot] = np.mean(received_powers_puea)

        # Print results for each time slot
        print(f"\nTime Slot {time_slot + 1}:")
        print(f"Average PU Received Power: {avg_pu_power_per_slot[time_slot]:.2f} dB")
        print(f"Average PUEA Received Power: {avg_puea_power_per_slot[time_slot]:.2f} dB")

        # Print detailed array of received powers from each source
        print("Array of received powers from Primary User:")
        print(received_powers_primary)
        print("Array of received powers from PUEA:")
        print(received_powers_puea)

        # Append to respective result matrices
        pu_result_matrix.append(received_powers_primary)
        puea_result_matrix.append(received_powers_puea)

        # Use the enhanced feature extraction function (now only returns the basic 5 features)
        # Calculate ONE feature vector per time slot by aggregating across all secondary users
        pu_enhanced_features = extract_enhanced_features(received_powers_primary)
        puea_enhanced_features = extract_enhanced_features(received_powers_puea)

        # Add the class labels (0 for PU, 1 for PUEA)
        pu_features.append(pu_enhanced_features + [0])  # Label 0 for PU
        puea_features.append(puea_enhanced_features + [1])  # Label 1 for PUEA

        # For printing the basic statistics (to maintain output format)
        pu_mean = pu_enhanced_features[0]
        pu_variance = pu_enhanced_features[1]
        pu_median = pu_enhanced_features[2]
        pu_lower_quartile = pu_enhanced_features[3]
        pu_upper_quartile = pu_enhanced_features[4]

        puea_mean = puea_enhanced_features[0]
        puea_variance = puea_enhanced_features[1]
        puea_median = puea_enhanced_features[2]
        puea_lower_quartile = puea_enhanced_features[3]
        puea_upper_quartile = puea_enhanced_features[4]

        # Print statistical features
        print("\nStatistical Features for PU:")
        print(f"  Mean: {pu_mean:.2f}, Variance: {pu_variance:.2f}, Median: {pu_median:.2f}")
        print(f"  Lower Quartile: {pu_lower_quartile:.2f}, Upper Quartile: {pu_upper_quartile:.2f}")

        print("\nStatistical Features for PUEA:")
        print(f"  Mean: {puea_mean:.2f}, Variance: {puea_variance:.2f}, Median: {puea_median:.2f}")
        print(f"  Lower Quartile: {puea_lower_quartile:.2f}, Upper Quartile: {puea_upper_quartile:.2f}")

    # Convert feature lists to numpy arrays for easier handling
    pu_features = np.array(pu_features)
    puea_features = np.array(puea_features)

    return pu_result_matrix, puea_result_matrix, avg_pu_power_per_slot, avg_puea_power_per_slot, pu_features, puea_features

# Main simulation parameters
primary_power = 20  # Constant value for primary user
puea_power = 30     # Fixed PUEA power for all scenarios

num_time_slots = 1000 # Number of time slots (iterations) per scenario

# Define the three scenarios with different PU and PUEA positions
scenarios = {
    'A': {
        'primary_position': np.array([20, 20]),
        'puea_position': np.array([80, 80]),
        'description': 'Far distance'
    },
    'B': {
        'primary_position': np.array([25, 25]),
        'puea_position': np.array([55, 55]),
        'description': 'Medium distance'
    },
    'C': {
        'primary_position': np.array([40, 40]),
        'puea_position': np.array([55, 55]),
        'description': 'Close distance'
    }
}


# Store results for different scenarios
scenario_results = {}
matrix_for_combining_scenarios = []

# Define percentages of PUEA energy labels to mix with PU labels
puea_percentages = [10, 20, 30, 40, 50]

# Store combined matrices for each case
scenario_case_matrices = {}

# Iterate through each scenario
for scenario_name, scenario_config in scenarios.items():
    print(f"\n\n========== SCENARIO {scenario_name}: {scenario_config['description']} ==========")

    primary_position = scenario_config['primary_position']
    puea_position = scenario_config['puea_position']

    pu_result, puea_result, avg_pu_per_slot, avg_puea_per_slot, pu_features, puea_features = run_simulation(
        primary_power,
        puea_power,
        num_time_slots,
        primary_position,
        puea_position,
        scenario_name
    )

    # Store results for original scenario
    scenario_results[scenario_name] = {
        'pu_result': pu_result,
        'puea_result': puea_result,
        'avg_pu_per_slot': avg_pu_per_slot,
        'avg_puea_per_slot': avg_puea_per_slot,
        'primary_position': primary_position,
        'puea_position': puea_position,
        'description': scenario_config['description'],
        'pu_features': pu_features,
        'puea_features': puea_features
    }

    print(f"\nCompleted simulation for Scenario {scenario_name}: {scenario_config['description']}")
    print(f"PU Features Matrix Shape: {pu_features.shape}")
    print(f"PUEA Features Matrix Shape: {puea_features.shape}")

    # Display the feature matrices
    print("\nPU Features Matrix (Rows: Time Slots, Columns: [Mean, Variance, Median, Lower Quartile, Upper Quartile]):")
    print(pu_features)

    print("\nPUEA Features Matrix (Rows: Time Slots, Columns: [Mean, Variance, Median, Lower Quartile, Upper Quartile]):")
    print(puea_features)

    # Combine the processed matrices into a single matrix for the base scenario
    combined_matrix = np.vstack((pu_features, puea_features))
    print("Combined Matrix for Base Scenario:")
    print(combined_matrix)
    matrix_for_combining_scenarios.append(combined_matrix)

    # Save the original combined matrix to a CSV file
    original_combined_file = f"{scenario_name}_original_combined_matrix.csv"
    columns = ['Mean', 'Variance', 'Median', 'Lower Quartile', 'Upper Quartile', 'Label']

    pd.DataFrame(combined_matrix, columns=columns).to_csv(original_combined_file, index=False)
    print(f"Saved original combined matrix for Scenario {scenario_name} to {original_combined_file}")

    # Now create 5 different cases for each scenario based on different percentages
    print(f"\nCreating different percentage cases for scenario {scenario_name}:")

    # Always use only the first num_time_slots PU and PUEA features
    pu_features_trimmed = pu_features[:num_time_slots]
    puea_features_trimmed = puea_features[:num_time_slots]

    for percentage in puea_percentages:
        num_puea_samples = max(1, int(round(num_time_slots * (percentage / 100))))
        num_pu_samples = num_time_slots - num_puea_samples

        # Shuffle before selecting to avoid always picking the same samples
        np.random.seed(int(percentage * 100 + ord(scenario_name[0])))
        pu_shuffled = pu_features_trimmed.copy()
        puea_shuffled = puea_features_trimmed.copy()
        np.random.shuffle(pu_shuffled)
        np.random.shuffle(puea_shuffled)

        selected_pu_features = pu_shuffled[:num_pu_samples]
        selected_puea_features = puea_shuffled[:num_puea_samples]

        # Combine and shuffle again
        case_combined_matrix = np.vstack((selected_pu_features, selected_puea_features))
        np.random.shuffle(case_combined_matrix)

        case_name = f"{scenario_name}_case_{percentage}percent"
        scenario_case_matrices[case_name] = case_combined_matrix

        output_file = f"{case_name}_matrix.csv"
        pd.DataFrame(case_combined_matrix, columns=columns).to_csv(output_file, index=False)

        print(f"  Created case {case_name} with {num_puea_samples} PUEA samples and {num_pu_samples} PU samples (total {num_time_slots})")
        print(f"  Saved combined matrix to {output_file}")

# Combine all original scenario matrices into one final matrix with gaps
final_matrix = []
for i, matrix in enumerate(matrix_for_combining_scenarios):
    # Add an identifier column to distinguish between matrices
    identifier_column = np.full((matrix.shape[0], 1), i)  # Identifier for each matrix
    matrix_with_id = np.hstack((matrix, identifier_column))
    final_matrix.append(matrix_with_id)

# Stack all matrices vertically
final_matrix = np.vstack(final_matrix)

# Save the final matrix to a CSV file
output_file = "final_matrix.csv"
columns = ['Mean', 'Variance', 'Median', 'Lower Quartile', 'Upper Quartile', 'Label', 'Matrix_ID']
pd.DataFrame(final_matrix, columns=columns).to_csv(output_file, index=False)

print(f"\nAll original scenario matrices combined and saved to {output_file}")

# Compare scenarios with scatter plot
plt.figure(figsize=(7, 5))

# Plot average PU and PUEA received powers for each scenario
for scenario_name, results in scenario_results.items():
    # Calculate distance between PU and PUEA
    distance = calculate_distance(results['primary_position'], results['puea_position'])

    # Calculate overall average powers across all time slots
    avg_pu = np.mean(results['avg_pu_per_slot'])
    avg_puea = np.mean(results['avg_puea_per_slot'])

    # Add points to the scatter plot
    plt.scatter(distance, avg_pu, label=f'PU - Scenario {scenario_name}',
                marker='o', s=100, edgecolors='black')
    plt.scatter(distance, avg_puea, label=f'PUEA - Scenario {scenario_name}',
                marker='s', s=100, edgecolors='black')

plt.title('Comparison of Scenarios: Effect of PU-PUEA Distance on Received Power')
plt.xlabel('Distance between PU and PUEA')
plt.ylabel('Average Received Power (dB)')
plt.grid(True)
plt.legend()
plt.tight_layout()
# plt.savefig('scenario_comparison.png')
plt.show()

# Output summary of statistical features
print("\n===== STATISTICAL FEATURES SUMMARY =====")
for scenario_name, results in scenario_results.items():
    distance = calculate_distance(results['primary_position'], results['puea_position'])
    print(f"\nScenario {scenario_name}: {results['description']} (PU-PUEA Distance: {distance:.2f} units)")

# Output summary of cases created
print("\n===== PERCENTAGE CASES SUMMARY =====")
for case_name in sorted(scenario_case_matrices.keys()):
    matrix = scenario_case_matrices[case_name]
    num_pu = np.sum(matrix[:, 5] == 0)
    num_puea = np.sum(matrix[:, 5] == 1)
    total = len(matrix)
    print(f"Case {case_name}: {num_pu} PU samples ({num_pu/total*100:.1f}%), {num_puea} PUEA samples ({num_puea/total*100:.1f}%), Total: {total}")

# Output overall summary
print("\n===== SIMULATION OVERALL SUMMARY =====")
print(f"Primary User Power: {primary_power} dB (constant)")
print(f"PUEA Power: {puea_power} dB (constant)")
print(f"Number of Time Slots per Scenario: {num_time_slots}")
print("\nScenario Descriptions:")

for scenario_name, results in scenario_results.items():
    distance = calculate_distance(results['primary_position'], results['puea_position'])
    print(f"\nScenario {scenario_name}: {results['description']}")
    print(f"  PU Position: ({results['primary_position'][0]}, {results['primary_position'][1]})")
    print(f"  PUEA Position: ({results['puea_position'][0]}, {results['puea_position'][1]})")
    print(f"  PU-PUEA Distance: {distance:.2f} units")

    # Calculate overall average powers across all time slots
    avg_pu = np.mean(results['avg_pu_per_slot'])
    avg_puea = np.mean(results['avg_puea_per_slot'])
    print(f"  Average PU Received Power: {avg_pu:.2f} dB")
    print(f"  Average PUEA Received Power: {avg_puea:.2f} dB")

print(f"\nTotal Cases Created: {len(scenario_case_matrices)}")
print(f"Percentages Used: {puea_percentages}")

In [ ]:
# Cell 2: Calculate Manhattan distance matrix for each case and scenario
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import StandardScaler
import os
import seaborn as sns
import matplotlib.pyplot as plt

# Define the cases and percentages (matching those from Cell 1)
cases = ['A', 'B', 'C']
percentages = [10, 20, 30, 40, 50]

# Function to calculate Manhattan distance matrix
def calculate_manhattan_distance(df):
    # Extract numerical features (excluding the Label column)
    features = df.iloc[:, :-1].values  # All columns except the last (Label)
    
    # Calculate pairwise Manhattan distances
    distances = pdist(features, metric='cityblock')
    
    # Convert to a square distance matrix
    distance_matrix = squareform(distances)
    
    return distance_matrix, features

# Create directory for storing distance matrices if it doesn't exist
distance_dir = "distance_matrices"
if not os.path.exists(distance_dir):
    os.makedirs(distance_dir)

# Dictionary to store distance matrices
distance_matrices = {}

# Process each case and scenario
for case in cases:
    for percentage in percentages:
        file_path = f"{case}_case_{percentage}percent_matrix.csv"
        
        try:
            # Read the scenario data
            df = pd.read_csv(file_path)
            print(f"Processing {file_path} with shape {df.shape}")
            
            # Calculate Manhattan distance matrix
            distance_matrix, features = calculate_manhattan_distance(df)
            
            # Round to 2 decimal places
            distance_matrix = np.round(distance_matrix, 2)
            
            # Store in dictionary with a key that identifies the case and scenario
            key = f"{case}_{percentage}"
            distance_matrices[key] = distance_matrix
            
            print(f"Calculated Manhattan distance matrix for {key} with shape {distance_matrix.shape}")
            
            # Save the distance matrix as CSV instead of NPY
            output_path = os.path.join(distance_dir, f"{case}_{percentage}_manhattan_dist.csv")
            # Create a DataFrame to save as CSV (index=False for cleaner output)
            pd.DataFrame(distance_matrix).to_csv(output_path, index=False)
            print(f"Saved distance matrix to {output_path}")
            
            # Also save a normalized version for visualization
            scaler = StandardScaler()
            normalized_features = scaler.fit_transform(features)
            normalized_distances = pdist(normalized_features, metric='cityblock')
            normalized_matrix = squareform(normalized_distances)
            
            # Round normalized matrix to 2 decimal places
            normalized_matrix = np.round(normalized_matrix, 2)
            
            # Save normalized matrix as CSV
            norm_output_path = os.path.join(distance_dir, f"{case}_{percentage}_normalized_dist.csv")
            pd.DataFrame(normalized_matrix).to_csv(norm_output_path, index=False)
            print(f"Saved normalized distance matrix to {norm_output_path}")
            
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")

# Display summary of processed matrices
print(f"\nProcessed {len(distance_matrices)} distance matrices")

# Display a sample of the first distance matrix (for verification)
if distance_matrices:
    sample_key = list(distance_matrices.keys())[0]
    print(f"\nSample of distance matrix for {sample_key}:")
    sample_matrix = distance_matrices[sample_key]
    print(sample_matrix[:5, :5])  # Display a 5x5 subset
    
    # Create heatmap visualization of the sample distance matrix
    plt.figure(figsize=(10, 8))
    plt.title(f"Manhattan Distance Matrix Heatmap - {sample_key}")
    sns.heatmap(sample_matrix[:20, :20], cmap="viridis", annot=False)
    plt.tight_layout()
    
    # Save the heatmap
    heatmap_path = os.path.join(distance_dir, f"{sample_key}_heatmap.png")
    plt.savefig(heatmap_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved heatmap visualization to {heatmap_path}")
else:
    print("No distance matrices were created. Please check file paths.")

In [ ]:
# # Cell 3: Improved clustering algorithms with better distance matrix utilization
# # 1st try (2d plotting) with PCA and MDS

# from sklearn.cluster import DBSCAN, AgglomerativeClustering, KMeans
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import adjusted_rand_score
# from sklearn.manifold import MDS

# # Dictionary to store clustering results
# clustering_results = {}

# # Function to perform DBSCAN clustering with the specified epsilon and minPoints calculation
# def perform_dbscan(distance_matrix, k=0.55, min_points=None):
#     """
#     Perform DBSCAN clustering with parameters based on the specified approach:
#     - minPoints (min_samples) is set to n/2 where n is the number of time slots (rows in distance matrix)
#     - epsilon is calculated as max(D) * k where D is the distance matrix
    
#     Parameters:
#         distance_matrix: Precomputed Manhattan distance matrix
#         k: Multiplier for epsilon calculation (default 0.5, can be tuned using ROC curves)
#         min_points: Override for min_points calculation (if None, will use n/2)
        
#     Returns:
#         Array of cluster labels for each data point
#     """
#     # Get number of time slots (n)
#     n = distance_matrix.shape[0]
    
#     # Set minPoints to n/2 as specified
#     if min_points is None:
#         min_points = max(2, int(n / 2))
#         print(f"Setting minPoints to n/2: {min_points}")
    
#     # Calculate epsilon as max(D) * k
#     # Get maximum distance value in the matrix
#     max_distance = np.max(distance_matrix)
#     epsilon = max_distance * k
#     print(f"Calculated epsilon = max(D) * k = {max_distance:.2f} * {k} = {epsilon:.2f}")
    
#     # Apply DBSCAN with calculated parameters
#     dbscan = DBSCAN(eps=epsilon, min_samples=min_points, metric='precomputed')
#     labels = dbscan.fit_predict(distance_matrix)
    
#     # Check number of clusters (excluding noise)
#     n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
#     print(f"DBSCAN produced {n_clusters} clusters with epsilon={epsilon:.2f}, minPoints={min_points}")
    
#     # Calculate percentage of points classified as noise
#     noise_percentage = 100 * np.sum(labels == -1) / len(labels)
#     print(f"Noise percentage: {noise_percentage:.2f}%")
    
#     # If we didn't get the expected 2 clusters, try different k values
#     if n_clusters != 2:
#         print(f"DBSCAN produced {n_clusters} clusters instead of 2, trying different k values...")
        
#         # Try a range of k values
#         k_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
        
#         for test_k in k_values:
#             if test_k == k:  # Skip the value we already tried
#                 continue
                
#             test_epsilon = max_distance * test_k
#             test_dbscan = DBSCAN(eps=test_epsilon, min_samples=min_points, metric='precomputed')
#             test_labels = test_dbscan.fit_predict(distance_matrix)
#             test_n_clusters = len(set(test_labels)) - (1 if -1 in test_labels else 0)
#             test_noise = 100 * np.sum(test_labels == -1) / len(test_labels)
            
#             print(f"  k={test_k}: epsilon={test_epsilon:.2f}, clusters={test_n_clusters}, noise={test_noise:.2f}%")
            
#             if test_n_clusters == 2:
#                 print(f"Found optimal k = {test_k} producing 2 clusters")
#                 return test_labels
    
#     # After all attempts, return the original labels
#     return labels

# # Function to perform Agglomerative clustering with improved parameter selection
# def perform_agglomerative(distance_matrix, n_clusters=2, linkage='average'):
#     # Try different linkage methods to find the best one
#     linkage_methods = ['average', 'complete', 'ward', 'single']
#     best_labels = None
#     best_silhouette = -1
    
#     for method in linkage_methods:
#         try:
#             # Ward linkage needs Euclidean distances, skip if using different metric
#             if method == 'ward' and distance_matrix.dtype != np.float64:
#                 continue
                
#             # Use the current linkage method
#             agg = AgglomerativeClustering(
#                 n_clusters=n_clusters, 
#                 linkage=method,
#                 metric='precomputed' if method != 'ward' else 'euclidean'
#             )
            
#             # For ward linkage, transform distances to euclidean representation if needed
#             if method == 'ward':
#                 # Use MDS to convert the distance matrix to euclidean space
#                 mds = MDS(n_components=min(5, distance_matrix.shape[0]-1), 
#                          dissimilarity='precomputed', random_state=42)
#                 euclidean_points = mds.fit_transform(distance_matrix)
#                 current_labels = agg.fit_predict(euclidean_points)
#             else:
#                 current_labels = agg.fit_predict(distance_matrix)
            
#             # Skip methods that produce only one cluster
#             if len(np.unique(current_labels)) < 2:
#                 continue
                
#             # Calculate silhouette score directly from distance matrix
#             from sklearn.metrics import silhouette_score
#             try:
#                 # For precomputed distances, we need to extract points
#                 mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
#                 points = mds.fit_transform(distance_matrix)
#                 current_silhouette = silhouette_score(points, current_labels)
                
#                 if current_silhouette > best_silhouette:
#                     best_silhouette = current_silhouette
#                     best_labels = current_labels
#                     print(f"Linkage method '{method}' has silhouette score: {current_silhouette:.4f}")
#             except:
#                 # If silhouette calculation fails, still keep the result
#                 if best_labels is None:
#                     best_labels = current_labels
#         except Exception as e:
#             print(f"Error with linkage method '{method}': {str(e)}")
    
#     # If no good method found, use average linkage as fallback
#     if best_labels is None:
#         try:
#             agg = AgglomerativeClustering(n_clusters=n_clusters, linkage='average', metric='precomputed')
#             best_labels = agg.fit_predict(distance_matrix)
#         except:
#             # Last resort: apply K-means on MDS coordinates
#             mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
#             pos = mds.fit_transform(distance_matrix)
#             kmeans = KMeans(n_clusters=n_clusters, random_state=42)
#             best_labels = kmeans.fit_predict(pos)
    
#     return best_labels

# # Function to perform K-means clustering with distance-aware initialization
# def perform_kmeans(distance_matrix, df, n_clusters=2):
#     # Use MDS to embed the distance matrix in a lower-dimensional Euclidean space
#     mds = MDS(n_components=min(5, distance_matrix.shape[0]-1), 
#              dissimilarity='precomputed', random_state=42)
#     points = mds.fit_transform(distance_matrix)
    
#     # Use the embedded points for K-means clustering
#     kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
#     labels = kmeans.fit_predict(points)
    
#     # Return the labels
#     return labels

# # Function to compute purity score (agreement with true labels when available)
# def compute_purity_score(labels, true_labels):
#     # Make sure cluster labels are comparable
#     from scipy.optimize import linear_sum_assignment
#     import numpy as np
    
#     # Create contingency matrix
#     contingency_matrix = np.zeros((np.max(labels) + 1, np.max(true_labels) + 1))
#     for i in range(len(labels)):
#         contingency_matrix[labels[i], true_labels[i]] += 1
        
#     # Find optimal one-to-one mapping between cluster and true labels
#     row_ind, col_ind = linear_sum_assignment(-contingency_matrix)
    
#     # Return purity score
#     return sum([contingency_matrix[row_ind[i], col_ind[i]] for i in range(len(row_ind))]) / len(labels)

# # Function to visualize clustering results
# def visualize_clustering(data, labels, title, original_labels=None):
#     # Apply PCA for visualization 
#     pca = PCA(n_components=2)
#     reduced_data = pca.fit_transform(data)
    
#     # Create a figure
#     plt.figure(figsize=(12, 10))
    
#     # Get unique labels
#     unique_labels = np.unique(labels)
    
#     # Plot each cluster
#     for label in unique_labels:
#         if label == -1:
#             # Noise points in black
#             mask = labels == label
#             plt.scatter(reduced_data[mask, 0], reduced_data[mask, 1], 
#                         c='black', marker='x', alpha=0.5, label='Noise')
#         else:
#             # Regular clusters
#             mask = labels == label
#             plt.scatter(reduced_data[mask, 0], reduced_data[mask, 1], 
#                         marker='o', alpha=0.7, label=f'Cluster {label}')
    
#     # If original labels are provided, add agreement metrics
#     if original_labels is not None:
#         # Calculate basic agreement (assuming binary labels)
#         if len(np.unique(labels)) == 2 and len(np.unique(original_labels)) == 2:
#             # If both have exactly 2 clusters, find best mapping and show agreement
#             # Consider both possible mappings and take the better one
#             mapping1 = {0:0, 1:1}
#             mapping2 = {0:1, 1:0}
            
#             # Map the cluster labels to the original labels using both mappings
#             mapped_labels1 = np.array([mapping1[l] if l in mapping1 else l for l in labels])
#             mapped_labels2 = np.array([mapping2[l] if l in mapping2 else l for l in labels])
            
#             # Calculate agreements
#             agreement1 = np.mean(mapped_labels1 == original_labels) * 100
#             agreement2 = np.mean(mapped_labels2 == original_labels) * 100
            
#             # Use the better mapping
#             agreement = max(agreement1, agreement2)
            
#             # Also calculate adjusted Rand Index for a more robust measure
#             ari = adjusted_rand_score(original_labels, labels)
            
#             title = f"{title}\nAgreement: {agreement:.1f}%, ARI: {ari:.3f}"
#         else:
#             # If not both binary, use ARI only
#             # Filter out noise points for the calculation
#             valid_indices = labels != -1
#             if np.sum(valid_indices) > 0:
#                 ari = adjusted_rand_score(original_labels[valid_indices], labels[valid_indices])
#                 title = f"{title}\nAdjusted Rand Index: {ari:.3f}"
    
#     plt.title(title, fontsize=14)
#     plt.xlabel('Principal Component 1', fontsize=12)
#     plt.ylabel('Principal Component 2', fontsize=12)
#     plt.legend(fontsize=10)
#     plt.grid(True, linestyle='--', alpha=0.7)
#     plt.tight_layout()
    
#     return plt

# # Create a directory for storing cluster visualizations
# cluster_viz_dir = "cluster_visualizations"
# if not os.path.exists(cluster_viz_dir):
#     os.makedirs(cluster_viz_dir)

# # Apply clustering to each case and scenario
# for case in cases:
#     for percentage in percentages:
#         key = f"{case}_{percentage}"
        
#         try:
#             # Get the distance matrix
#             distance_matrix = distance_matrices[key]
            
#             # Read the original data
#             file_path = f"{case}_case_{percentage}percent_matrix.csv"
#             df = pd.read_csv(file_path)
            
#             # Extract features and standardize for visualization
#             features = df.select_dtypes(include=['float64', 'int64']).values
#             scaler = StandardScaler()
#             scaled_features = scaler.fit_transform(features)
            
#             # Get original labels if available (for calculating agreement)
#             original_labels = None
#             if 'Label' in df.columns:
#                 original_labels = df['Label'].values
            
#             print(f"\nClustering {key}...")
            
#             # Perform DBSCAN with optimal eps calculation
#             print(f"Running DBSCAN on {key}...")
#             dbscan_labels = perform_dbscan(distance_matrix)
            
#             # Perform Agglomerative Clustering with multiple linkage methods
#             print(f"Running Agglomerative clustering on {key}...")
#             agg_labels = perform_agglomerative(distance_matrix)
            
#             # Perform K-means on MDS embedding of distance matrix
#             print(f"Running K-means on {key}...")
#             kmeans_labels = perform_kmeans(distance_matrix, df)
            
#             # Store results
#             clustering_results[f"{key}_dbscan"] = dbscan_labels
#             clustering_results[f"{key}_agglomerative"] = agg_labels
#             clustering_results[f"{key}_kmeans"] = kmeans_labels
            
#             # Print cluster distribution
#             print(f"\nCluster distribution for {key}:")
#             print(f"DBSCAN: {np.bincount(dbscan_labels + 1) if -1 in dbscan_labels else np.bincount(dbscan_labels)}")
#             print(f"Agglomerative: {np.bincount(agg_labels)}")
#             print(f"K-means: {np.bincount(kmeans_labels)}")
            
#             # Check if clusters match expected count
#             dbscan_n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
#             print(f"Number of clusters - DBSCAN: {dbscan_n_clusters}, Agglomerative: {len(set(agg_labels))}, K-means: {len(set(kmeans_labels))}")
            
#             # If original labels exist, calculate agreement metrics
#             if original_labels is not None:
#                 # Calculate Adjusted Rand Index (ARI) for each method
#                 # For DBSCAN, exclude noise points for agreement calculation
#                 if -1 in dbscan_labels:
#                     valid_indices = dbscan_labels != -1
#                     valid_dbscan = dbscan_labels[valid_indices]
#                     valid_original = original_labels[valid_indices]
#                     dbscan_ari = adjusted_rand_score(valid_original, valid_dbscan) if len(valid_dbscan) > 0 else 0
#                 else:
#                     dbscan_ari = adjusted_rand_score(original_labels, dbscan_labels)
                
#                 agg_ari = adjusted_rand_score(original_labels, agg_labels)
#                 kmeans_ari = adjusted_rand_score(original_labels, kmeans_labels)
                
#                 print(f"Adjusted Rand Index - DBSCAN: {dbscan_ari:.3f}, Agglomerative: {agg_ari:.3f}, K-means: {kmeans_ari:.3f}")
            
#             # Visualize clustering results
#             # DBSCAN
#             dbscan_plot = visualize_clustering(scaled_features, dbscan_labels, 
#                                             f"DBSCAN Clustering - Case {case}, {percentage}% Data", 
#                                             original_labels)
#             dbscan_plot.savefig(f"{cluster_viz_dir}/{key}_dbscan_clustering.png", dpi=300, bbox_inches='tight')
#             plt.close()
            
#             # Agglomerative
#             agg_plot = visualize_clustering(scaled_features, agg_labels, 
#                                         f"Agglomerative Clustering - Case {case}, {percentage}% Data",
#                                         original_labels)
#             agg_plot.savefig(f"{cluster_viz_dir}/{key}_agglomerative_clustering.png", dpi=300, bbox_inches='tight')
#             plt.close()
            
#             # K-means
#             kmeans_plot = visualize_clustering(scaled_features, kmeans_labels, 
#                                             f"K-means Clustering - Case {case}, {percentage}% Data",
#                                             original_labels)
#             kmeans_plot.savefig(f"{cluster_viz_dir}/{key}_kmeans_clustering.png", dpi=300, bbox_inches='tight')
#             plt.close()
            
#             print(f"Visualizations saved for {key} in {cluster_viz_dir} directory")
            
#         except KeyError:
#             print(f"Distance matrix not found for {key}")
#         except Exception as e:
#             print(f"Error clustering {key}: {str(e)}")

# # Add this code at the end of Cell 3 (before the summary print statements)
# print("\nSaving clustering results to CSV files for performance evaluation...")

# # Create directory if it doesn't exist
# if not os.path.exists(cluster_viz_dir):
#     os.makedirs(cluster_viz_dir)

# # Save clustering results to CSV files
# for case in cases:
#     for percentage in percentages:
#         key = f"{case}_{percentage}"
        
#         # Get the original data
#         file_path = f"{case}_case_{percentage}percent_matrix.csv"
        
#         try:
#             # Check if the file exists
#             if os.path.exists(file_path):
#                 # Load the data to get the true labels if available
#                 df = pd.read_csv(file_path)
                
#                 # Check if we have clustering results for this case/percentage
#                 dbscan_key = f"{key}_dbscan"
#                 kmeans_key = f"{key}_kmeans"
#                 agg_key = f"{key}_agglomerative"
                
#                 # Choose one of the clustering results (prioritize k-means, then agglomerative, then DBSCAN)
#                 if kmeans_key in clustering_results:
#                     selected_labels = clustering_results[kmeans_key]
#                     selected_algorithm = "K-means"
#                 elif agg_key in clustering_results:
#                     selected_labels = clustering_results[agg_key]
#                     selected_algorithm = "Agglomerative"
#                 elif dbscan_key in clustering_results:
#                     selected_labels = clustering_results[dbscan_key]
#                     selected_algorithm = "DBSCAN"
#                 else:
#                     print(f"No clustering results found for {key}")
#                     continue
                
#                 # Create DataFrame with the clustering results
#                 result_df = pd.DataFrame({
#                     'Cluster': selected_labels
#                 })
                
#                 # Add the original labels if available
#                 if 'Label' in df.columns:
#                     result_df['True_Label'] = df['Label'].values
                
#                 # Save to CSV
#                 output_path = f"{cluster_viz_dir}/{key}_clustering_result.csv"
#                 result_df.to_csv(output_path, index=False)
#                 print(f"Saved clustering result for {key} ({selected_algorithm}) to {output_path}")
                
#         except Exception as e:
#             print(f"Error saving clustering result for {key}: {str(e)}")

# print("Clustering results saved successfully.")

# # Summary of clustering results
# print(f"\nCompleted clustering for {len(clustering_results) // 3} case-scenario combinations")
# print(f"Generated {len(clustering_results)} clustering results (3 algorithms per case-scenario)")
# print(f"Visualization files saved in {cluster_viz_dir}")

In [ ]:
# # Enhanced clustering for PUEA detection with multiple algorithms 
# 2nd try (3d plotting) with PCA and MDS


# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
# from sklearn.metrics import adjusted_rand_score, silhouette_score
# from sklearn.preprocessing import StandardScaler
# import os

# # Create output directories (properly creating nested directories)
# output_dir = "clustering_results"
# algorithm_dirs = ["kmeans", "dbscan", "agglomerative"]

# # Create main directory
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# # Create algorithm-specific subdirectories
# for alg_dir in algorithm_dirs:
#     dir_path = os.path.join(output_dir, alg_dir)
#     if not os.path.exists(dir_path):
#         os.makedirs(dir_path)

# # Define cases and percentages
# cases = ['A', 'B', 'C']
# percentages = [10, 20, 30, 40, 50]

# def perform_kmeans_clustering(features, original_labels=None):
#     """Performs K-means clustering on the feature space"""
#     kmeans = KMeans(n_clusters=2, n_init=10, random_state=42)
#     labels = kmeans.fit_predict(features)
    
#     results = analyze_clustering_results(labels, original_labels, "K-means")
#     return results

# def perform_dbscan_clustering(features, original_labels=None):
#     """Performs DBSCAN clustering on the feature space"""
#     # Normalize features for better DBSCAN performance
#     scaler = StandardScaler()
#     normalized_features = scaler.fit_transform(features)
    
#     # Try different eps values until we get close to 2 clusters
#     best_eps = 0.5
#     best_labels = None
#     best_n_clusters = 0
    
#     for eps in [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
#         dbscan = DBSCAN(eps=eps, min_samples=5)
#         current_labels = dbscan.fit_predict(normalized_features)
#         n_clusters = len(set(current_labels)) - (1 if -1 in current_labels else 0)
        
#         # We want 2 clusters, but will accept 1-3 clusters
#         if n_clusters >= 1 and n_clusters <= 3:
#             best_eps = eps
#             best_labels = current_labels
#             best_n_clusters = n_clusters
#             if n_clusters == 2:
#                 break
    
#     if best_labels is None:
#         # Fallback if DBSCAN fails to find reasonable clusters
#         kmeans = KMeans(n_clusters=2, n_init=10, random_state=42)
#         best_labels = kmeans.fit_predict(features)
#         print("DBSCAN failed to find reasonable clusters. Falling back to K-means.")
    
#     # If DBSCAN found noise points (-1), assign them to the nearest cluster
#     if -1 in best_labels:
#         noise_indices = np.where(best_labels == -1)[0]
#         if len(noise_indices) > 0:
#             # For each noise point, find the nearest non-noise cluster
#             for idx in noise_indices:
#                 # Use a simple distance to cluster centroid approach
#                 min_dist = float('inf')
#                 assigned_cluster = 0
                
#                 for cluster in set(best_labels) - {-1}:
#                     cluster_points = normalized_features[best_labels == cluster]
#                     if len(cluster_points) > 0:
#                         # Calculate mean as cluster centroid
#                         centroid = np.mean(cluster_points, axis=0)
#                         dist = np.sum((normalized_features[idx] - centroid) ** 2)
#                         if dist < min_dist:
#                             min_dist = dist
#                             assigned_cluster = cluster
                
#                 best_labels[idx] = assigned_cluster
    
#     # If DBSCAN found only one cluster or more than two clusters, remap labels to 0/1
#     unique_labels = set(best_labels)
#     if len(unique_labels) != 2:
#         if len(unique_labels) > 2:
#             # Keep the two largest clusters
#             sizes = [(label, np.sum(best_labels == label)) for label in unique_labels]
#             top_two = sorted(sizes, key=lambda x: x[1], reverse=True)[:2]
            
#             # Create a mapping from original labels to 0/1
#             label_map = {top_two[0][0]: 0, top_two[1][0]: 1}
            
#             # Apply the mapping
#             new_labels = np.array([label_map.get(label, -1) for label in best_labels])
            
#             # Assign all other labels to the closest of the top two clusters
#             for label in unique_labels:
#                 if label not in label_map:
#                     mask = best_labels == label
#                     if np.any(mask):
#                         # Assign to the nearest of the two main clusters
#                         points = normalized_features[mask]
#                         centroid0 = np.mean(normalized_features[best_labels == top_two[0][0]], axis=0)
#                         centroid1 = np.mean(normalized_features[best_labels == top_two[1][0]], axis=0)
                        
#                         dist0 = np.mean(np.sum((points - centroid0) ** 2, axis=1))
#                         dist1 = np.mean(np.sum((points - centroid1) ** 2, axis=1))
                        
#                         new_labels[mask] = 0 if dist0 < dist1 else 1
            
#             best_labels = new_labels
#         else:
#             # If there's only one cluster, split using k-means
#             print("DBSCAN found only one cluster. Splitting with K-means.")
#             kmeans = KMeans(n_clusters=2, n_init=10, random_state=42)
#             best_labels = kmeans.fit_predict(features)
    
#     results = analyze_clustering_results(best_labels, original_labels, "DBSCAN")
#     return results

# def perform_agglomerative_clustering(features, original_labels=None):
#     """Performs Agglomerative clustering on the feature space"""
#     # Normalize features
#     scaler = StandardScaler()
#     normalized_features = scaler.fit_transform(features)
    
#     # Try different linkage criteria
#     best_labels = None
#     best_silhouette = -1
    
#     for linkage in ['ward', 'complete', 'average', 'single']:
#         try:
#             agg = AgglomerativeClustering(n_clusters=2, linkage=linkage)
#             current_labels = agg.fit_predict(normalized_features)
            
#             # Calculate silhouette score if possible
#             if len(set(current_labels)) > 1:
#                 try:
#                     silhouette = silhouette_score(normalized_features, current_labels)
#                     if silhouette > best_silhouette:
#                         best_silhouette = silhouette
#                         best_labels = current_labels
#                         print(f"Agglomerative with {linkage} linkage: silhouette = {silhouette:.4f}")
#                 except Exception as e:
#                     print(f"Error calculating silhouette for {linkage}: {str(e)}")
#         except Exception as e:
#             print(f"Error with {linkage} linkage: {str(e)}")
    
#     if best_labels is None:
#         # Fallback if all linkage methods fail
#         agg = AgglomerativeClustering(n_clusters=2)
#         best_labels = agg.fit_predict(normalized_features)
    
#     results = analyze_clustering_results(best_labels, original_labels, "Agglomerative")
#     return results

# def analyze_clustering_results(labels, original_labels=None, algorithm_name=""):
#     """Analyzes clustering results and calculates performance metrics"""
#     # Count cluster sizes
#     count_0 = np.sum(labels == 0)
#     count_1 = np.sum(labels == 1)
#     total = len(labels)
    
#     # Calculate actual PUEA percentage from ground truth
#     actual_puea_percentage = None
#     if original_labels is not None:
#         actual_puea_percentage = np.mean(original_labels == 1) * 100
        
#         # If we have original labels, possibly flip clusters for better alignment
#         agreement1 = np.mean(labels == original_labels) * 100
#         swapped_labels = 1 - labels
#         agreement2 = np.mean(swapped_labels == original_labels) * 100
        
#         if agreement2 > agreement1:
#             labels = swapped_labels
#             agreement = agreement2
#         else:
#             agreement = agreement1
        
#         ari = adjusted_rand_score(original_labels, labels)
#         print(f"{algorithm_name} - Agreement with original labels: {agreement:.2f}%, ARI: {ari:.4f}")
        
#         # Determine which cluster is predominantly PUEA vs PU
#         puea_in_cluster0 = np.sum((labels == 0) & (original_labels == 1))
#         puea_in_cluster1 = np.sum((labels == 1) & (original_labels == 1))
#         pu_in_cluster0 = np.sum((labels == 0) & (original_labels == 0))
#         pu_in_cluster1 = np.sum((labels == 1) & (original_labels == 0))
        
#         # Identify PUEA cluster
#         puea_cluster = 1 if puea_in_cluster1 > puea_in_cluster0 else 0
#         pu_cluster = 0 if puea_cluster == 1 else 1
        
#         # Calculate true counts
#         total_pu = np.sum(original_labels == 0)
#         total_puea = np.sum(original_labels == 1)
        
#         # Calculate detection rates
#         puea_detection_rate = np.sum((labels == puea_cluster) & (original_labels == 1)) / total_puea if total_puea > 0 else 0
#         false_detection_rate = np.sum((labels == puea_cluster) & (original_labels == 0)) / total_pu if total_pu > 0 else 0
        
#         print(f"{algorithm_name} - PUEA Detection Rate: {puea_detection_rate:.4f}")
#         print(f"{algorithm_name} - False Detection Rate: {false_detection_rate:.4f}")
        
#         # Return results dictionary
#         results = {
#             'labels': labels,
#             'count_0': count_0, 
#             'count_1': count_1,
#             'puea_cluster': puea_cluster,
#             'pu_cluster': pu_cluster,
#             'puea_detection_rate': puea_detection_rate,
#             'false_detection_rate': false_detection_rate,
#             'accuracy': agreement / 100,
#             'ari': ari,
#             'actual_puea_percentage': actual_puea_percentage,
#             'pu_in_cluster0': pu_in_cluster0,
#             'pu_in_cluster1': pu_in_cluster1,
#             'puea_in_cluster0': puea_in_cluster0,
#             'puea_in_cluster1': puea_in_cluster1,
#             'total_pu': total_pu,
#             'total_puea': total_puea
#         }
#     else:
#         # Return basic results if no ground truth
#         results = {
#             'labels': labels,
#             'count_0': count_0, 
#             'count_1': count_1
#         }
    
#     return results

# def visualize_clustering_results(features, results, case, percentage, algorithm, output_dir):
#     """Creates 2D and 3D visualizations of clustering results"""
#     # Fix algorithm folder name to lowercase for consistent paths
#     alg_folder = algorithm.lower()
    
#     # Create 2D plot (mean vs variance)
#     plt.figure(figsize=(10, 8))
#     plt.title(f"{algorithm} Clustering - Case {case}, {percentage}% Scenario", fontsize=14)
#     plt.xlabel('Mean', fontsize=12)
#     plt.ylabel('Variance', fontsize=12)
    
#     # Plot each cluster with different colors
#     for cluster_label in [0, 1]:
#         mask = results['labels'] == cluster_label
#         plt.scatter(features[mask, 0], features[mask, 1],
#                    label=f'Cluster {cluster_label}', alpha=0.7)
    
#     # Add detection metrics in title if available
#     if 'puea_detection_rate' in results:
#         plt.title(f"{algorithm} Clustering - Case {case}, {percentage}% Scenario\n" +
#                  f"PUEA Detection Rate: {results['puea_detection_rate']:.2f}, " +
#                  f"False Detection Rate: {results['false_detection_rate']:.2f}, " +
#                  f"Accuracy: {results['accuracy']:.2f}")
    
#     plt.grid(True)
#     plt.legend()
    
#     # Save 2D plot
#     plt.savefig(f"{output_dir}/{alg_folder}/{case}_{percentage}_2d_clustering.png", 
#                dpi=300, bbox_inches='tight')
#     plt.close()
    
#     # Create 3D plot (median, lower quartile, upper quartile)
#     fig = plt.figure(figsize=(12, 10))
#     ax = fig.add_subplot(111, projection='3d')
#     ax.set_title(f"{algorithm} 3D Clustering - Case {case}, {percentage}% Scenario", fontsize=14)
#     ax.set_xlabel('Median', fontsize=12)
#     ax.set_ylabel('Lower Quartile', fontsize=12)
#     ax.set_zlabel('Upper Quartile', fontsize=12)

#     # Plot each cluster in 3D
#     for cluster_label in [0, 1]:
#         mask = results['labels'] == cluster_label
#         if np.any(mask):  # Only plot if there are points in this cluster
#             scatter = ax.scatter(
#                 features[mask, 2],  # Median (index 2)
#                 features[mask, 3],  # Lower Quartile (index 3)
#                 features[mask, 4],  # Upper Quartile (index 4)
#                 label=f'Cluster {cluster_label}',
#                 alpha=0.7,
#                 s=50
#             )
    
#     # Add detection metrics in title if available
#     if 'puea_detection_rate' in results:
#         ax.set_title(f"{algorithm} 3D Clustering - Case {case}, {percentage}% Scenario\n" +
#                  f"PUEA Detection Rate: {results['puea_detection_rate']:.2f}, " +
#                  f"False Detection Rate: {results['false_detection_rate']:.2f}")
    
#     # Add legend and adjust view
#     ax.legend()
#     ax.view_init(elev=30, azim=45)
#     plt.tight_layout()
    
#     # Save 3D plot
#     plt.savefig(f"{output_dir}/{alg_folder}/{case}_{percentage}_3d_clustering.png", 
#                dpi=300, bbox_inches='tight')
#     plt.close()

# # Process each case and scenario
# all_results = {
#     'kmeans': [],
#     'dbscan': [],
#     'agglomerative': []
# }

# for case in cases:
#     for percentage in percentages:
#         key = f"{case}_{percentage}"
#         print(f"\n\nProcessing {key}...")
        
#         try:
#             # Load data
#             file_path = f"{case}_case_{percentage}percent_matrix.csv"
#             df = pd.read_csv(file_path)
            
#             # Extract features and labels
#             features = df.iloc[:, :-1].values
#             original_labels = df['Label'].values if 'Label' in df.columns else None
            
#             print(f"Loaded dataset with {features.shape[0]} samples, {features.shape[1]} features")
            
#             # Perform K-means clustering
#             print("\nPerforming K-means clustering...")
#             kmeans_results = perform_kmeans_clustering(features, original_labels)
            
#             # Perform DBSCAN clustering
#             print("\nPerforming DBSCAN clustering...")
#             dbscan_results = perform_dbscan_clustering(features, original_labels)
            
#             # Perform Agglomerative clustering
#             print("\nPerforming Agglomerative clustering...")
#             agglomerative_results = perform_agglomerative_clustering(features, original_labels)
            
#             # Visualize results for each algorithm
#             print("\nGenerating visualizations...")
#             visualize_clustering_results(features, kmeans_results, case, percentage, 
#                                         "K-means", output_dir)
#             visualize_clustering_results(features, dbscan_results, case, percentage, 
#                                         "DBSCAN", output_dir)
#             visualize_clustering_results(features, agglomerative_results, case, percentage, 
#                                         "Agglomerative", output_dir)
            
#             # Save results to DataFrame if original labels are available
#             if original_labels is not None:
#                 # K-means results
#                 all_results['kmeans'].append({
#                     'Case': case,
#                     'Percentage': percentage,
#                     'Actual_PUEA_Percentage': kmeans_results['actual_puea_percentage'],
#                     'PUEA_Detection_Rate': kmeans_results['puea_detection_rate'],
#                     'False_Detection_Rate': kmeans_results['false_detection_rate'],
#                     'Accuracy': kmeans_results['accuracy'],
#                     'ARI': kmeans_results['ari'],
#                     'PUEA_Cluster': kmeans_results['puea_cluster'],
#                     'Cluster0_Size': kmeans_results['count_0'],
#                     'Cluster1_Size': kmeans_results['count_1']
#                 })
                
#                 # DBSCAN results
#                 all_results['dbscan'].append({
#                     'Case': case,
#                     'Percentage': percentage,
#                     'Actual_PUEA_Percentage': dbscan_results['actual_puea_percentage'],
#                     'PUEA_Detection_Rate': dbscan_results['puea_detection_rate'],
#                     'False_Detection_Rate': dbscan_results['false_detection_rate'],
#                     'Accuracy': dbscan_results['accuracy'],
#                     'ARI': dbscan_results['ari'],
#                     'PUEA_Cluster': dbscan_results['puea_cluster'],
#                     'Cluster0_Size': dbscan_results['count_0'],
#                     'Cluster1_Size': dbscan_results['count_1']
#                 })
                
#                 # Agglomerative results
#                 all_results['agglomerative'].append({
#                     'Case': case,
#                     'Percentage': percentage,
#                     'Actual_PUEA_Percentage': agglomerative_results['actual_puea_percentage'],
#                     'PUEA_Detection_Rate': agglomerative_results['puea_detection_rate'],
#                     'False_Detection_Rate': agglomerative_results['false_detection_rate'],
#                     'Accuracy': agglomerative_results['accuracy'],
#                     'ARI': agglomerative_results['ari'],
#                     'PUEA_Cluster': agglomerative_results['puea_cluster'],
#                     'Cluster0_Size': agglomerative_results['count_0'],
#                     'Cluster1_Size': agglomerative_results['count_1']
#                 })
                
#         except Exception as e:
#             print(f"Error processing {key}: {str(e)}")

# # Save all results to CSV
# for algorithm, results in all_results.items():
#     if results:
#         results_df = pd.DataFrame(results)
#         results_df.to_csv(f"{output_dir}/{algorithm}_clustering_summary.csv", index=False)
#         print(f"\nResults saved to {output_dir}/{algorithm}_clustering_summary.csv")

# # Create a 3D feature space visualization showing all features for each case and percentage
# for case in cases:
#     for percentage in percentages:
#         try:
#             file_path = f"{case}_case_{percentage}percent_matrix.csv"
#             if not os.path.exists(file_path):
#                 continue
                
#             df = pd.read_csv(file_path)
#             features = df.iloc[:, :-1].values
#             labels = df['Label'].values if 'Label' in df.columns else None
            
#             if labels is not None:
#                 plt.figure(figsize=(14, 12))
#                 ax = plt.axes(projection='3d')
                
#                 # Plot PU points (label 0)
#                 pu_mask = labels == 0
#                 ax.scatter3D(
#                     features[pu_mask, 2],     # Median
#                     features[pu_mask, 3],     # Lower Quartile
#                     features[pu_mask, 4],     # Upper Quartile
#                     c='blue',
#                     marker='o',
#                     label='Primary User (PU)',
#                     alpha=0.7,
#                     s=50
#                 )
                
#                 # Plot PUEA points (label 1)
#                 puea_mask = labels == 1
#                 ax.scatter3D(
#                     features[puea_mask, 2],   # Median
#                     features[puea_mask, 3],   # Lower Quartile
#                     features[puea_mask, 4],   # Upper Quartile
#                     c='red',
#                     marker='^',
#                     label='PUEA',
#                     alpha=0.7,
#                     s=50
#                 )
                
#                 # Add labels and title
#                 ax.set_xlabel('Median', fontsize=12)
#                 ax.set_ylabel('Lower Quartile', fontsize=12)
#                 ax.set_zlabel('Upper Quartile', fontsize=12)
#                 ax.set_title(f'3D Feature Space - Case {case}, {percentage}% PUEA', 
#                              fontsize=14)
#                 ax.legend(fontsize=12)
                
#                 # Adjust view angle
#                 ax.view_init(elev=30, azim=45)
                
#                 # Save the 3D feature visualization
#                 plt.savefig(f"{output_dir}/3d_feature_space_{case}_{percentage}.png", dpi=300, bbox_inches='tight')
#                 plt.close()
#                 print(f"\n3D feature space visualization saved for Case {case}, {percentage}% PUEA")
#         except Exception as e:
#             print(f"Error creating 3D feature space for {case}_{percentage}: {str(e)}")

# # Create comparative detection rate visualization
# if all(len(all_results[alg]) > 0 for alg in ['kmeans', 'dbscan', 'agglomerative']):
#     plt.figure(figsize=(18, 12))
    
#     # Create 3x2 grid of subplots (3 cases, 2 metrics each)
#     for i, case in enumerate(cases):
#         # PUEA Detection Rate plot
#         plt.subplot(3, 2, 2*i+1)
        
#         # Plot for each algorithm
#         for algorithm, results in all_results.items():
#             case_data = pd.DataFrame(results)
#             case_data = case_data[case_data['Case'] == case]
#             if not case_data.empty:
#                 plt.plot(case_data['Percentage'], case_data['PUEA_Detection_Rate'], 
#                         marker='o', linewidth=2, label=f"{algorithm.capitalize()}")
        
#         plt.title(f'Case {case} - PUEA Detection Rate', fontsize=14)
#         plt.xlabel('PUEA Percentage in Dataset (%)', fontsize=12)
#         plt.ylabel('Detection Rate', fontsize=12)
#         plt.grid(True)
#         plt.xticks(percentages)
#         plt.ylim(0, 1.05)
#         if i == 0:  # Add legend only to the first row
#             plt.legend(loc='lower right')
        
#         # False Detection Rate plot
#         plt.subplot(3, 2, 2*i+2)
        
#         # Plot for each algorithm
#         for algorithm, results in all_results.items():
#             case_data = pd.DataFrame(results)
#             case_data = case_data[case_data['Case'] == case]
#             if not case_data.empty:
#                 plt.plot(case_data['Percentage'], case_data['False_Detection_Rate'], 
#                         marker='x', linestyle='--', linewidth=2, label=f"{algorithm.capitalize()}")
        
#         plt.title(f'Case {case} - False Detection Rate', fontsize=14)
#         plt.xlabel('PUEA Percentage in Dataset (%)', fontsize=12)
#         plt.ylabel('False Detection Rate', fontsize=12)
#         plt.grid(True)
#         plt.xticks(percentages)
#         plt.ylim(0, 1.05)
#         if i == 0:  # Add legend only to the first row
#             plt.legend(loc='upper right')
    
#     plt.tight_layout()
#     plt.subplots_adjust(top=0.92)
#     plt.suptitle('Comparative PUEA Detection Performance Across All Algorithms', fontsize=16)
    
#     plt.savefig(f"{output_dir}/comparative_detection_performance.png", dpi=300, bbox_inches='tight')
#     plt.close()
#     print(f"\nComparative detection performance visualization saved to {output_dir}/comparative_detection_performance.png")

# print("\nClustering analysis completed with multiple algorithms!")

In [ ]:
    # Cell 3: Improved clustering algorithms with better distance matrix utilization
    from sklearn.cluster import DBSCAN, AgglomerativeClustering, KMeans
    import matplotlib.pyplot as plt
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import adjusted_rand_score, silhouette_score
    from sklearn.manifold import MDS
    from sklearn.neighbors import NearestNeighbors, kneighbors_graph
    import numpy as np
    import pandas as pd
    import os
    from collections import Counter

    # Dictionary to store clustering results
    clustering_results = {}

    # Function to perform DBSCAN clustering with KNN refinement
    def perform_dbscan(distance_matrix, k=0.4, min_points=None):
        """
        Perform DBSCAN clustering with parameters based on the specified approach:
        - minPoints (min_samples) is set to n/2 where n is the number of time slots (rows in distance matrix)
        - epsilon is calculated as max(D) * k where D is the distance matrix
        
        Parameters:
            distance_matrix: Precomputed Manhattan distance matrix
            k: Multiplier for epsilon calculation (default 0.4, can be tuned)
            min_points: Override for min_points calculation (if None, will use n/2)
            
        Returns:
            Array of cluster labels for each data point
        """
        # Get number of time slots (n)
        n = distance_matrix.shape[0]
        
        # Set minPoints to n/2 as specified
        if min_points is None:
            min_points = max(2, int(n / 2))
            print(f"Setting minPoints to n/2: {min_points}")
        
        # Calculate epsilon as max(D) * k
        max_distance = np.max(distance_matrix)
        epsilon = max_distance * k
        print(f"Calculated epsilon = max(D) * k = {max_distance:.2f} * {k} = {epsilon:.2f}")
        
        # Apply DBSCAN with calculated parameters
        dbscan = DBSCAN(eps=epsilon, min_samples=min_points, metric='precomputed')
        labels = dbscan.fit_predict(distance_matrix)
        
        # Check number of clusters (excluding noise)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        noise_percentage = 100 * np.sum(labels == -1) / len(labels)
        print(f"DBSCAN produced {n_clusters} clusters with epsilon={epsilon:.2f}, minPoints={min_points}")
        print(f"Noise percentage: {noise_percentage:.2f}%")
        
        # If we didn't get the expected 2 clusters or have too much noise, try different k values
        if n_clusters != 2 or noise_percentage > 30:
            print(f"DBSCAN produced {n_clusters} clusters with {noise_percentage:.2f}% noise, trying different parameters...")
            
            # Try a range of k values and min_points adjustments
            k_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
            min_points_factors = [0.25, 0.33, 0.5, 0.75]  # Different fractions of n
            
            best_labels = labels
            best_n_clusters = n_clusters
            best_noise = noise_percentage
            
            for test_k in k_values:
                for mp_factor in min_points_factors:
                    test_min_points = max(2, int(n * mp_factor))
                    if test_k == k and test_min_points == min_points:  # Skip the value we already tried
                        continue
                        
                    test_epsilon = max_distance * test_k
                    test_dbscan = DBSCAN(eps=test_epsilon, min_samples=test_min_points, metric='precomputed')
                    test_labels = test_dbscan.fit_predict(distance_matrix)
                    test_n_clusters = len(set(test_labels)) - (1 if -1 in test_labels else 0)
                    test_noise = 100 * np.sum(test_labels == -1) / len(test_labels)
                    
                    # We want exactly 2 clusters with lower noise
                    if test_n_clusters == 2 and test_noise < best_noise:
                        best_labels = test_labels
                        best_n_clusters = 2
                        best_noise = test_noise
                        print(f"  Found better parameters: k={test_k}, min_points={test_min_points}, noise={test_noise:.2f}%")
                        
                        # If we find a good solution with low noise, we can stop
                        if test_noise < 15:
                            print(f"  Found excellent solution with low noise: {test_noise:.2f}%")
                            labels = test_labels
                            n_clusters = test_n_clusters
                            break
                else:
                    continue
                break
        
        # Apply KNN refinement to improve cluster boundaries
        if -1 in labels or n_clusters != 2:
            print("Applying KNN refinement to improve clustering...")
            
            # Use MDS to convert distance matrix to points in Euclidean space for KNN
            mds = MDS(n_components=min(5, n-1), dissimilarity='precomputed', random_state=42)
            points = mds.fit_transform(distance_matrix)
            
            # Choose k for KNN (dynamically based on dataset size)
            knn_k = min(15, max(5, int(n/10)))
            print(f"Using k={knn_k} for KNN refinement")
            
            # Create a KNN graph
            nbrs = NearestNeighbors(n_neighbors=knn_k).fit(points)
            distances, indices = nbrs.kneighbors(points)
            
            # Handle noise points first
            refined_labels = np.copy(labels)
            
            # Assign noise points to clusters based on nearest neighbors
            if -1 in labels:
                noise_indices = np.where(labels == -1)[0]
                for i in noise_indices:
                    # Get neighbors that aren't noise
                    valid_neighbors = [idx for idx in indices[i] if labels[idx] != -1]
                    if valid_neighbors:
                        # Assign to most common class among neighbors
                        neighbor_labels = [labels[idx] for idx in valid_neighbors]
                        most_common = Counter(neighbor_labels).most_common(1)[0][0]
                        refined_labels[i] = most_common
            
            # Refine edge points (points that might be misclassified)
            for i in range(len(points)):
                # Get labels of neighbors
                neighbor_labels = [refined_labels[idx] for idx in indices[i]]
                counts = Counter(neighbor_labels)
                # If majority of neighbors have different label, change it
                if counts.most_common(1)[0][0] != refined_labels[i] and counts.most_common(1)[0][1] > knn_k/2:
                    refined_labels[i] = counts.most_common(1)[0][0]
            
            # Ensure we have exactly 2 clusters now
            unique_labels = list(set(refined_labels))
            if len(unique_labels) == 2:
                print("KNN refinement successful - achieved 2 clusters")
                
                # Map unique labels to 0 and 1 if needed
                if unique_labels != [0, 1]:
                    new_labels = np.zeros_like(refined_labels)
                    new_labels[refined_labels == unique_labels[1]] = 1
                    return new_labels
                return refined_labels
                
            elif len(unique_labels) > 2:
                print(f"KNN refinement produced {len(unique_labels)} clusters - merging to get 2")
                
                # Merge smaller clusters into the two largest ones
                cluster_sizes = [(label, np.sum(refined_labels == label)) for label in unique_labels]
                top_two = sorted(cluster_sizes, key=lambda x: x[1], reverse=True)[:2]
                main_clusters = [top_two[0][0], top_two[1][0]]
                
                # Create final labels
                final_labels = np.zeros_like(refined_labels)
                final_labels[refined_labels == main_clusters[1]] = 1
                
                # Assign remaining clusters to closest main cluster
                for label in unique_labels:
                    if label not in main_clusters:
                        mask = refined_labels == label
                        if np.any(mask):
                            # Find closest main cluster
                            pts = points[mask]
                            centroid0 = np.mean(points[refined_labels == main_clusters[0]], axis=0)
                            centroid1 = np.mean(points[refined_labels == main_clusters[1]], axis=0)
                            
                            dist0 = np.mean(np.sum((pts - centroid0)**2, axis=1))
                            dist1 = np.mean(np.sum((pts - centroid1)**2, axis=1))
                            
                            assigned_cluster = 0 if dist0 < dist1 else 1
                            final_labels[mask] = assigned_cluster
                
                return final_labels
            else:
                # If KNN created only 1 cluster, force 2 clusters with K-means
                print("KNN refinement produced only 1 cluster - using K-means to create 2")
                kmeans = KMeans(n_clusters=2, random_state=42)
                final_labels = kmeans.fit_predict(points)
                return final_labels
        
        # If we already have 2 clusters, just normalize labels to 0 and 1
        if n_clusters == 2:
            unique_non_noise = sorted([l for l in set(labels) if l != -1])
            if len(unique_non_noise) == 2 and (unique_non_noise != [0, 1]):
                final_labels = np.zeros_like(labels)
                final_labels[labels == unique_non_noise[1]] = 1
                return final_labels
        
        return labels

    # Enhanced Agglomerative clustering with improved KNN refinement
    def perform_agglomerative(distance_matrix, n_clusters=2, linkage='average'):
        # Try different linkage methods with more sophisticated evaluation
        linkage_methods = ['average', 'complete', 'ward', 'single']
        best_labels = None
        best_score = -float('inf')  # Combined quality metric
        best_method = None
        
        for method in linkage_methods:
            try:
                # Ward linkage needs Euclidean distances
                if method == 'ward':
                    # Use MDS with higher dimensions to get better Euclidean representation
                    mds = MDS(n_components=min(8, distance_matrix.shape[0]-1), 
                            dissimilarity='precomputed', random_state=42,
                            n_init=3, max_iter=300)  # Multiple initializations, more iterations
                    euclidean_points = mds.fit_transform(distance_matrix)
                    current_labels = AgglomerativeClustering(
                        n_clusters=n_clusters, 
                        linkage=method
                    ).fit_predict(euclidean_points)
                else:
                    # Use precomputed distance matrix directly
                    current_labels = AgglomerativeClustering(
                        n_clusters=n_clusters, 
                        linkage=method,
                        metric='precomputed'
                    ).fit_predict(distance_matrix)
                
                # Skip methods that produce only one cluster
                if len(np.unique(current_labels)) < 2:
                    continue
                    
                # Calculate multiple evaluation metrics for better method selection
                try:
                    # For precomputed distances, we need to extract points
                    mds_eval = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
                    points = mds_eval.fit_transform(distance_matrix)
                    
                    # Calculate silhouette score
                    silhouette = silhouette_score(points, current_labels)
                    
                    # Calculate Davies-Bouldin Index (lower is better)
                    from sklearn.metrics import davies_bouldin_score
                    davies_bouldin = -davies_bouldin_score(points, current_labels)  # Negate so higher is better
                    
                    # Calculate cluster compactness (intra-cluster distances)
                    intra_cluster_distances = []
                    for cluster_id in range(n_clusters):
                        cluster_points = points[current_labels == cluster_id]
                        if len(cluster_points) > 1:
                            centroid = np.mean(cluster_points, axis=0)
                            avg_distance = np.mean([np.sum((pt - centroid) ** 2) for pt in cluster_points])
                            intra_cluster_distances.append(avg_distance)
                    
                    compactness = -np.mean(intra_cluster_distances) if intra_cluster_distances else 0  # Negate so higher is better
                    
                    # Calculate balanced cluster size score (penalize highly imbalanced clusters)
                    cluster_sizes = [np.sum(current_labels == i) for i in range(n_clusters)]
                    balance = -np.std(cluster_sizes) / np.mean(cluster_sizes) if np.mean(cluster_sizes) > 0 else -1.0
                    
                    # Combined score (weighted average of metrics)
                    combined_score = (0.5 * silhouette + 
                                    0.2 * davies_bouldin + 
                                    0.2 * compactness + 
                                    0.1 * balance)
                    
                    print(f"  Linkage '{method}': silhouette={silhouette:.3f}, davies={davies_bouldin:.3f}, balance={balance:.3f}, combined={combined_score:.3f}")
                    
                    if combined_score > best_score:
                        best_score = combined_score
                        best_labels = current_labels
                        best_method = method
                        print(f"  Found new best method: {method} with score {combined_score:.4f}")
                    
                except Exception as e:
                    print(f"  Error calculating metrics for {method}: {str(e)}")
                    # If metric calculation fails but we don't have any labels yet, keep these
                    if best_labels is None:
                        best_labels = current_labels
                        best_method = method
            except Exception as e:
                print(f"  Error with linkage method '{method}': {str(e)}")
        
        # If no good method found, use average linkage as fallback
        if best_labels is None:
            print("No suitable linkage method found. Using average linkage as fallback.")
            try:
                agg = AgglomerativeClustering(n_clusters=n_clusters, linkage='average', metric='precomputed')
                best_labels = agg.fit_predict(distance_matrix)
                best_method = 'average'
            except Exception:
                # Last resort: apply K-means on MDS coordinates
                print("Fallback to K-means on MDS coordinates")
                mds = MDS(n_components=3, dissimilarity='precomputed', random_state=42)
                points = mds.fit_transform(distance_matrix)
                kmeans = KMeans(n_clusters=n_clusters, n_init=15, random_state=42)
                best_labels = kmeans.fit_predict(points)
        
        print(f"Selected linkage method: {best_method}")
        
        # Apply enhanced multi-scale KNN refinement to improve cluster boundaries
        print("Applying enhanced KNN refinement to Agglomerative clustering results...")
        
        # Store original agglomerative labels for ensemble
        original_labels = np.copy(best_labels)
        
        # Get number of data points
        n = distance_matrix.shape[0]
        
        # Create points for KNN refinement with higher dimensionality
        mds_points = MDS(n_components=min(8, n-1), dissimilarity='precomputed', random_state=42)
        points = mds_points.fit_transform(distance_matrix)
        
        # Apply multi-scale refinement with different neighborhood sizes
        knn_scales = [
            max(3, int(n/25)),        # Very local neighborhood
            max(7, int(n/15)),        # Local neighborhood
            min(20, max(10, int(n/8))) # Extended neighborhood
        ]
        print(f"Using multi-scale KNN refinement with scales: {knn_scales}")
        
        refined_labels = np.copy(original_labels)
        confidence_map = np.ones(n)  # Confidence in current label assignment
        
        # First pass: Progressive refinement at increasing scales
        for knn_k in knn_scales:
            # Create KNN graph
            nbrs = NearestNeighbors(n_neighbors=knn_k).fit(points)
            distances, indices = nbrs.kneighbors(points)
            
            # Normalize distances for weighting
            distances = distances / (np.max(distances) if np.max(distances) > 0 else 1.0)
            weights = 1.0 - distances  # Closer neighbors have higher weight
            
            # Refine boundary points using distance-weighted KNN voting
            for i in range(n):
                # Skip high-confidence points at larger scales
                if knn_k > 10 and confidence_map[i] > 0.85:
                    continue
                    
                # Get weighted votes from neighbors
                weighted_votes = {0: 0.0, 1: 0.0}
                
                for j, idx in enumerate(indices[i]):
                    weighted_votes[refined_labels[idx]] = weighted_votes.get(refined_labels[idx], 0.0) + weights[i, j]
                    
                # Normalize the votes
                total_votes = sum(weighted_votes.values())
                if total_votes > 0:
                    for label in weighted_votes:
                        weighted_votes[label] /= total_votes
                    
                    # Get majority label and confidence
                    majority_label = max(weighted_votes.items(), key=lambda x: x[1])[0]
                    confidence = weighted_votes[majority_label]
                    
                    # Only change if confidence is high enough and different from current
                    if confidence > 0.65 and majority_label != refined_labels[i]:
                        refined_labels[i] = majority_label
                        confidence_map[i] = confidence
        
        # Second pass: Apply graph-based spectral clustering for smoother boundaries
        # Create KNN similarity graph
        optimal_k = min(25, max(15, int(n/6)))
        knn_graph = kneighbors_graph(points, n_neighbors=optimal_k, mode='distance')
        
        # Convert distances to similarities
        sigma = np.median(knn_graph.data) * 1.5  # Adaptive kernel width based on median distance
        knn_graph.data = np.exp(-knn_graph.data**2 / (2. * sigma**2))
        
        # Initialize spectral clustering with refined labels as guidance
        from sklearn.cluster import SpectralClustering
        spectral = SpectralClustering(n_clusters=2, affinity='precomputed', 
                                    assign_labels='kmeans', random_state=42,
                                    n_init=10)  # Multiple initializations
        spectral_labels = spectral.fit_predict(knn_graph)
        
        # Check if spectral labels match refined labels
        agreement = np.mean(refined_labels == spectral_labels)
        if agreement < 0.5:
            # If the labels are significantly different, invert the spectral labels
            spectral_labels = 1 - spectral_labels
            agreement = np.mean(refined_labels == spectral_labels)
        
        print(f"Agreement between KNN-refined and spectral labels: {agreement:.3f}")
        
        # Third pass: Create ensemble of different methods
        # Apply consensus voting between original, refined, and spectral
        ensemble_votes = np.zeros((n, n_clusters))
        
        # Add votes from each method with different weights
        # Original agglomerative labels (weight 1.0)
        for i, label in enumerate(original_labels):
            ensemble_votes[i, label] += 1.0
            
        # KNN refined labels (weight 1.2)
        for i, label in enumerate(refined_labels):
            ensemble_votes[i, label] += 1.2
        
        # Spectral labels (weight 0.8)
        for i, label in enumerate(spectral_labels):
            ensemble_votes[i, label] += 0.8
        
        # Final labels based on weighted voting
        final_labels = np.argmax(ensemble_votes, axis=1)
        
        # Check if the ensemble inverted the labels compared to original
        if np.mean(final_labels != original_labels) > 0.5:
            # If more than half the points changed, probably the clusters got inverted
            # Flip them back for consistency with original labels
            final_labels = 1 - final_labels
            
        # Calculate how many points changed from original agglomerative
        changes = np.sum(final_labels != original_labels)
        print(f"Agglomerative clustering with enhanced refinement complete. {changes} points ({changes/n*100:.1f}%) were refined.")
            
        return final_labels

    # Enhanced K-means clustering with improved KNN refinement
    def perform_kmeans(distance_matrix, df=None, n_clusters=2):
        """
        Perform K-means clustering with enhanced KNN refinement for better boundary detection
        
        Parameters:
            distance_matrix: Precomputed Manhattan distance matrix
            df: Original dataframe (optional, used for feature information)
            n_clusters: Number of clusters to identify (default 2 for PU vs PUEA)
            
        Returns:
            Array of cluster labels for each data point
        """
        # Get number of data points
        n = distance_matrix.shape[0]
        
        # Use MDS to embed the distance matrix in a higher dimensional Euclidean space
        print(f"Converting distance matrix to Euclidean space using MDS...")
        # Increase dimension from 5 to 8 for better embedding and preservation of distance relationships
        mds = MDS(n_components=min(8, n-1), dissimilarity='precomputed', random_state=42, 
                n_init=3, max_iter=500)  # Multiple initializations and more iterations
        points = mds.fit_transform(distance_matrix)
        
        # Apply K-means with multiple initializations and more iterations for stability
        print(f"Performing K-means clustering with n_clusters={n_clusters}...")
        kmeans = KMeans(n_clusters=n_clusters, n_init=20, max_iter=500, random_state=42)
        initial_labels = kmeans.fit_predict(points)
        
        # Apply multi-level KNN refinement to improve cluster boundaries
        print("Applying enhanced KNN refinement to K-means results...")
        
        # Store original points and labels for later comparison
        original_labels = np.copy(initial_labels)
        
        # Use multiple k values for KNN to capture different scales of neighborhood structure
        knn_k_values = [
            max(3, int(n/20)),     # Small neighborhood
            min(15, max(5, int(n/10))),  # Medium neighborhood
            min(30, max(10, int(n/5)))   # Large neighborhood
        ]
        print(f"Using multi-scale KNN refinement with k values: {knn_k_values}")
        
        # Initialize refined labels
        refined_labels = np.copy(initial_labels)
        
        # First pass: Apply KNN refinement at different scales with confidence weighting
        confidence_scores = np.ones(n)  # Initialize confidence scores
        
        for knn_k in knn_k_values:
            # Create KNN graph for current scale
            nbrs = NearestNeighbors(n_neighbors=knn_k).fit(points)
            distances, indices = nbrs.kneighbors(points)
            
            # Normalize distances for weighting
            max_dist = np.max(distances) if np.max(distances) > 0 else 1.0
            normalized_distances = distances / max_dist
            
            # Calculate distance-weighted votes for each point
            for i in range(n):
                # Get labels of neighbors with distance-based weights
                neighbor_weights = 1.0 - normalized_distances[i]  # Closer neighbors get higher weight
                
                # Get labels and their weights
                neighbor_labels = [refined_labels[idx] for idx in indices[i]]
                
                # Count weighted votes for each label
                label_votes = {}
                for j, label in enumerate(neighbor_labels):
                    if label not in label_votes:
                        label_votes[label] = 0.0
                    label_votes[label] += neighbor_weights[j]
                
                # Find label with highest weighted vote
                max_vote_label = max(label_votes.items(), key=lambda x: x[1])[0]
                
                # Calculate confidence in this classification
                total_votes = sum(label_votes.values())
                winning_vote_strength = label_votes[max_vote_label] / total_votes if total_votes > 0 else 0.0
                
                # Only change label if the winning vote is strong enough
                if winning_vote_strength > 0.65 and max_vote_label != refined_labels[i]:
                    refined_labels[i] = max_vote_label
                    confidence_scores[i] = winning_vote_strength
        
        # Use weighted features if dataframe is provided (combining feature space and distance space)
        if df is not None and 'Mean' in df.columns and 'Variance' in df.columns:
            # Extract key features for PU vs PUEA discrimination - give more weight to discriminative features
            features = df[['Mean', 'Variance', 'Median', 'Lower Quartile', 'Upper Quartile']].values
            
            # Standardize features
            scaler = StandardScaler()
            scaled_features = scaler.fit_transform(features)
            
            # Apply PCA to extract most discriminative directions
            pca = PCA(n_components=min(3, scaled_features.shape[1]))
            pca_features = pca.fit_transform(scaled_features)
            print(f"PCA explained variance ratio: {pca.explained_variance_ratio_}")
            
            # Create cluster centroids in feature space
            centroid0 = np.mean(pca_features[refined_labels == 0], axis=0)
            centroid1 = np.mean(pca_features[refined_labels == 1], axis=0)
            
            # Identify ambiguous points (points near cluster boundaries)
            ambiguous_scores = np.zeros(n)
            for i in range(n):
                # Calculate distance to both centroids
                dist0 = np.sum((pca_features[i] - centroid0) ** 2)
                dist1 = np.sum((pca_features[i] - centroid1) ** 2)
                
                # Ambiguity score: how close the distances to both centroids are
                total_dist = dist0 + dist1
                if total_dist > 0:
                    if refined_labels[i] == 0:
                        ambiguous_scores[i] = dist0 / total_dist  # Lower is more confident
                    else:
                        ambiguous_scores[i] = dist1 / total_dist  # Lower is more confident
            
            # Focus on ambiguous points (points that might be misclassified based on feature space)
            ambiguous_threshold = 0.4  # Points with ambiguity above this are suspicious
            ambiguous_indices = np.where(ambiguous_scores > ambiguous_threshold)[0]
            
            if len(ambiguous_indices) > 0:
                print(f"Found {len(ambiguous_indices)} ambiguous points to refine using feature space")
                
                # Use large neighborhood for these ambiguous points
                largest_k = min(50, max(30, int(n/4)))
                nbrs_large = NearestNeighbors(n_neighbors=largest_k).fit(points)
                large_distances, large_indices = nbrs_large.kneighbors(points)
                
                for i in ambiguous_indices:
                    # Get weighted votes from neighbors in larger neighborhood
                    weights = 1.0 - large_distances[i] / np.max(large_distances[i])
                    neighbor_labels = [refined_labels[idx] for idx in large_indices[i]]
                    
                    # Calculate weighted votes
                    votes = {0: 0.0, 1: 0.0}
                    for j, label in enumerate(neighbor_labels):
                        votes[label] += weights[j]
                    
                    # Assign to class with higher weighted vote
                    if votes[0] > votes[1]:
                        refined_labels[i] = 0
                    else:
                        refined_labels[i] = 1
        
        # Final pass: Ensemble with spectral clustering for smoother boundaries
        print("Applying spectral clustering for final refinement...")
        
        # Create KNN similarity graph with optimal parameters
        optimal_k = min(20, max(10, int(n/8)))
        knn_graph = kneighbors_graph(points, n_neighbors=optimal_k, mode='distance')
        
        # Convert distances to similarities with Gaussian kernel
        sigma = np.mean(knn_graph.data) * 2  # Adaptive sigma based on mean distance
        knn_graph.data = np.exp(-knn_graph.data**2 / (2. * sigma**2))
        
        # Use spectral clustering for final refinement
        from sklearn.cluster import SpectralClustering
        spectral = SpectralClustering(n_clusters=2, affinity='precomputed', 
                                    assign_labels='kmeans', random_state=42, 
                                    n_init=10)  # Multiple initializations
        spectral_labels = spectral.fit_predict(knn_graph)
        
        # Create ensemble of all methods
        ensemble_votes = np.zeros((n, n_clusters))
        
        # Add votes from each method with different weights
        # Original K-means labels (weight 1.0)
        for i, label in enumerate(original_labels):
            ensemble_votes[i, label] += 1.0
            
        # Refined KNN labels (weight 1.5)
        for i, label in enumerate(refined_labels):
            ensemble_votes[i, label] += 1.5
            
        # Spectral labels (weight 1.2)
        for i, label in enumerate(spectral_labels):
            ensemble_votes[i, label] += 1.2
        
        # Final labels based on majority voting
        final_labels = np.argmax(ensemble_votes, axis=1)
        
        # Check if we need to flip the labels to maintain consistency
        if np.mean(final_labels != original_labels) > 0.5:
            final_labels = 1 - final_labels
        
        # Calculate how many points changed from original K-means
        changes = np.sum(final_labels != original_labels)
        print(f"K-means with enhanced KNN refinement complete. {changes} points ({changes/n*100:.1f}%) were refined.")
        
        return final_labels

    # Function to compute purity score (agreement with true labels when available)
    def compute_purity_score(labels, true_labels):
        # Make sure cluster labels are comparable
        from scipy.optimize import linear_sum_assignment
        
        # Create contingency matrix
        contingency_matrix = np.zeros((np.max(labels) + 1, np.max(true_labels) + 1))
        for i in range(len(labels)):
            contingency_matrix[labels[i], true_labels[i]] += 1
            
        # Find optimal one-to-one mapping between cluster and true labels
        row_ind, col_ind = linear_sum_assignment(-contingency_matrix)
        
        # Return purity score
        return sum([contingency_matrix[row_ind[i], col_ind[i]] for i in range(len(row_ind))]) / len(labels)

    # Function to visualize clustering results
    def visualize_clustering(data, labels, title, original_labels=None):
        # Apply PCA for visualization 
        pca = PCA(n_components=2)
        reduced_data = pca.fit_transform(data)
        
        # Create a figure
        plt.figure(figsize=(12, 10))
        
        # Get unique labels
        unique_labels = np.unique(labels)
        
        # Plot each cluster
        for label in unique_labels:
            if label == -1:
                # Noise points in black
                mask = labels == label
                plt.scatter(reduced_data[mask, 0], reduced_data[mask, 1], 
                            c='black', marker='x', alpha=0.5, label='Noise')
            else:
                # Regular clusters
                mask = labels == label
                plt.scatter(reduced_data[mask, 0], reduced_data[mask, 1], 
                            marker='o', alpha=0.7, label=f'Cluster {label}')
        
        # If original labels are provided, add agreement metrics
        if original_labels is not None:
            # Calculate basic agreement (assuming binary labels)
            if len(np.unique(labels)) == 2 and len(np.unique(original_labels)) == 2:
                # If both have exactly 2 clusters, find best mapping and show agreement
                # Consider both possible mappings and take the better one
                mapping1 = {0:0, 1:1}
                mapping2 = {0:1, 1:0}
                
                # Map the cluster labels to the original labels using both mappings
                mapped_labels1 = np.array([mapping1[l] if l in mapping1 else l for l in labels])
                mapped_labels2 = np.array([mapping2[l] if l in mapping2 else l for l in labels])
                
                # Calculate agreements
                agreement1 = np.mean(mapped_labels1 == original_labels) * 100
                agreement2 = np.mean(mapped_labels2 == original_labels) * 100
                
                # Use the better mapping
                agreement = max(agreement1, agreement2)
                
                # Also calculate adjusted Rand Index for a more robust measure
                ari = adjusted_rand_score(original_labels, labels)
                
                title = f"{title}\nAgreement: {agreement:.1f}%, ARI: {ari:.3f}"
            else:
                # If not both binary, use ARI only
                # Filter out noise points for the calculation
                valid_indices = labels != -1
                if np.sum(valid_indices) > 0:
                    ari = adjusted_rand_score(original_labels[valid_indices], labels[valid_indices])
                    title = f"{title}\nAdjusted Rand Index: {ari:.3f}"
        
        plt.title(title, fontsize=14)
        plt.xlabel('Principal Component 1', fontsize=12)
        plt.ylabel('Principal Component 2', fontsize=12)
        plt.legend(fontsize=10)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.tight_layout()
        
        return plt

    # Create a directory for storing cluster visualizations
    cluster_viz_dir = "cluster_visualizations"
    if not os.path.exists(cluster_viz_dir):
        os.makedirs(cluster_viz_dir)

    # Define the cases and percentages
    cases = ['A', 'B', 'C']
    percentages = [10, 20, 30, 40, 50]

    # Apply clustering to each case and scenario
    for case in cases:
        for percentage in percentages:
            key = f"{case}_{percentage}"
            
            try:
                # Get the distance matrix
                distance_matrix = distance_matrices[key]
                
                # Read the original data
                file_path = f"{case}_case_{percentage}percent_matrix.csv"
                df = pd.read_csv(file_path)
                
                # Extract features and standardize for visualization
                features = df.select_dtypes(include=['float64', 'int64']).values
                scaler = StandardScaler()
                scaled_features = scaler.fit_transform(features)
                
                # Get original labels if available (for calculating agreement)
                original_labels = None
                if 'Label' in df.columns:
                    original_labels = df['Label'].values
                
                print(f"\nClustering {key}...")
                
                # Perform DBSCAN with optimal eps calculation and KNN refinement
                print(f"Running DBSCAN with KNN refinement on {key}...")
                dbscan_labels = perform_dbscan(distance_matrix)
                
                # Perform Agglomerative Clustering with multiple linkage methods and KNN refinement
                print(f"Running Agglomerative clustering with KNN refinement on {key}...")
                agg_labels = perform_agglomerative(distance_matrix)
                
                # Perform K-means with MDS embedding and KNN refinement
                print(f"Running K-means with KNN refinement on {key}...")
                kmeans_labels = perform_kmeans(distance_matrix, df)
                
                # Store results
                clustering_results[f"{key}_dbscan"] = dbscan_labels
                clustering_results[f"{key}_agglomerative"] = agg_labels
                clustering_results[f"{key}_kmeans"] = kmeans_labels
                
                # Print cluster distribution
                print(f"\nCluster distribution for {key}:")
                print(f"DBSCAN: {np.bincount(dbscan_labels + 1) if -1 in dbscan_labels else np.bincount(dbscan_labels)}")
                print(f"Agglomerative: {np.bincount(agg_labels)}")
                print(f"K-means: {np.bincount(kmeans_labels)}")
                
                # Check if clusters match expected count
                dbscan_n_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
                print(f"Number of clusters - DBSCAN: {dbscan_n_clusters}, Agglomerative: {len(set(agg_labels))}, K-means: {len(set(kmeans_labels))}")
                
                # If original labels exist, calculate agreement metrics
                if original_labels is not None:
                    # Calculate Adjusted Rand Index (ARI) for each method
                    # For DBSCAN, exclude noise points for agreement calculation
                    if -1 in dbscan_labels:
                        valid_indices = dbscan_labels != -1
                        valid_dbscan = dbscan_labels[valid_indices]
                        valid_original = original_labels[valid_indices]
                        dbscan_ari = adjusted_rand_score(valid_original, valid_dbscan) if len(valid_dbscan) > 0 else 0
                    else:
                        dbscan_ari = adjusted_rand_score(original_labels, dbscan_labels)
                    
                    agg_ari = adjusted_rand_score(original_labels, agg_labels)
                    kmeans_ari = adjusted_rand_score(original_labels, kmeans_labels)
                    
                    print(f"Adjusted Rand Index - DBSCAN: {dbscan_ari:.3f}, Agglomerative: {agg_ari:.3f}, K-means: {kmeans_ari:.3f}")
                
                # Visualize clustering results
                # DBSCAN
                dbscan_plot = visualize_clustering(scaled_features, dbscan_labels, 
                                                f"DBSCAN Clustering with KNN Refinement - Case {case}, {percentage}% Data", 
                                                original_labels)
                dbscan_plot.savefig(f"{cluster_viz_dir}/{key}_dbscan_clustering.png", dpi=300, bbox_inches='tight')
                plt.close()
                
                # Agglomerative
                agg_plot = visualize_clustering(scaled_features, agg_labels, 
                                            f"Agglomerative Clustering with KNN Refinement - Case {case}, {percentage}% Data",
                                            original_labels)
                agg_plot.savefig(f"{cluster_viz_dir}/{key}_agglomerative_clustering.png", dpi=300, bbox_inches='tight')
                plt.close()
                
                # K-means
                kmeans_plot = visualize_clustering(scaled_features, kmeans_labels, 
                                                f"K-means Clustering with KNN Refinement - Case {case}, {percentage}% Data",
                                                original_labels)
                kmeans_plot.savefig(f"{cluster_viz_dir}/{key}_kmeans_clustering.png", dpi=300, bbox_inches='tight')
                plt.close()
                
                print(f"Visualizations saved for {key} in {cluster_viz_dir} directory")
                
            except KeyError:
                print(f"Distance matrix not found for {key}")
            except Exception as e:
                print(f"Error clustering {key}: {str(e)}")

    # Save clustering results to CSV files for performance evaluation
    print("\nSaving clustering results to CSV files for performance evaluation...")

    # Create directory if it doesn't exist
    if not os.path.exists(cluster_viz_dir):
        os.makedirs(cluster_viz_dir)

    # Save clustering results to CSV files
    for case in cases:
        for percentage in percentages:
            key = f"{case}_{percentage}"
            
            # Get the original data
            file_path = f"{case}_case_{percentage}percent_matrix.csv"
            
            try:
                # Check if the file exists
                if os.path.exists(file_path):
                    # Load the data to get the true labels if available
                    df = pd.read_csv(file_path)
                    
                    # Check if we have clustering results for this case/percentage
                    dbscan_key = f"{key}_dbscan"
                    kmeans_key = f"{key}_kmeans"
                    agg_key = f"{key}_agglomerative"
                    
                    # Choose one of the clustering results (prioritize k-means, then agglomerative, then DBSCAN)
                    if kmeans_key in clustering_results:
                        selected_labels = clustering_results[kmeans_key]
                        selected_algorithm = "K-means"
                    elif agg_key in clustering_results:
                        selected_labels = clustering_results[agg_key]
                        selected_algorithm = "Agglomerative"
                    elif dbscan_key in clustering_results:
                        selected_labels = clustering_results[dbscan_key]
                        selected_algorithm = "DBSCAN"
                    else:
                        print(f"No clustering results found for {key}")
                        continue
                    
                    # Create DataFrame with the clustering results
                    result_df = pd.DataFrame({
                        'Cluster': selected_labels
                    })
                    
                    # Add the original labels if available
                    if 'Label' in df.columns:
                        result_df['True_Label'] = df['Label'].values
                    
                    # Save to CSV
                    output_path = f"{cluster_viz_dir}/{key}_clustering_result.csv"
                    result_df.to_csv(output_path, index=False)
                    print(f"Saved clustering result for {key} ({selected_algorithm}) to {output_path}")
                    
            except Exception as e:
                print(f"Error saving clustering result for {key}: {str(e)}")

    print("Clustering results saved successfully.")

    # Summary of clustering results
    print(f"\nCompleted clustering for {len(clustering_results) // 3} case-scenario combinations")
    print(f"Generated {len(clustering_results)} clustering results (3 algorithms per case-scenario)")
    print(f"Visualization files saved in {cluster_viz_dir}")

In [ ]:
# Compare detection rates and create combined performance metrics
print("\n=== Comparing Detection Rates and Combined Performance Metrics ===")

# Create a directory for performance visualizations
perf_viz_dir = "performance_visualizations"
if not os.path.exists(perf_viz_dir):
    os.makedirs(perf_viz_dir)

# Dictionary to store detection metrics for all algorithms
detection_metrics = {
    'case': [],
    'percentage': [],
    'algorithm': [],
    'puea_detection_rate': [],
    'false_detection_rate': [],
    'accuracy': [],
    'ari': []
}

# Add debug info to track what results we actually have
print("\nAvailable clustering results keys:")
print(sorted(list(clustering_results.keys())))

# Process each case and percentage
for case in cases:
    for percentage in percentages:
        key = f"{case}_{percentage}"
        print(f"\nProcessing metrics for {key}...")
        
        try:
            # Load the original data
            file_path = f"{case}_case_{percentage}percent_matrix.csv"
            if not os.path.exists(file_path):
                print(f"  File not found: {file_path}")
                continue
                
            df = pd.read_csv(file_path)
            true_labels = df['Label'].values if 'Label' in df.columns else None
            
            if true_labels is None:
                print(f"  No ground truth labels found in {file_path}")
                continue
                
            # Calculate detection metrics for each algorithm
            for alg_name, alg_key in [
                ('DBSCAN', f"{key}_dbscan"),
                ('K-means', f"{key}_kmeans"),
                ('Agglomerative', f"{key}_agglomerative")
            ]:
                print(f"  Processing algorithm: {alg_name}, key: {alg_key}")
                if alg_key in clustering_results:
                    labels = clustering_results[alg_key]
                    
                    # Check that the labels have the expected format
                    unique_labels = np.unique(labels)
                    print(f"    Found {len(unique_labels)} unique labels: {unique_labels}")
                    
                    # For DBSCAN, ignore noise points (-1) in the calculation
                    valid_indices = np.ones_like(labels, dtype=bool)
                    if -1 in labels:
                        valid_indices = labels != -1
                        if not np.any(valid_indices):
                            print(f"    All points classified as noise for {alg_name}, skipping")
                            continue
                    
                    valid_labels = labels[valid_indices]
                    valid_true = true_labels[valid_indices]
                    
                    # Count distribution of items in each cluster
                    pu_counts = [0, 0]  # Count of PU items in clusters 0 and 1
                    puea_counts = [0, 0]  # Count of PUEA items in clusters 0 and 1
                    
                    # Ensure that labels are 0 and 1
                    if set(np.unique(valid_labels)) != {0, 1}:
                        print(f"    Warning: Labels are not 0 and 1, remapping: {np.unique(valid_labels)}")
                        # Remap labels to 0 and 1
                        if len(np.unique(valid_labels)) == 2:
                            label_map = {u: i for i, u in enumerate(sorted(np.unique(valid_labels)))}
                            valid_labels = np.array([label_map[l] for l in valid_labels])
                            print(f"    Remapped labels: {np.unique(valid_labels)}")
                        else:
                            print(f"    Cannot remap labels, found {len(np.unique(valid_labels))} unique values")
                            continue
                    
                    for i in range(len(valid_labels)):
                        if valid_true[i] == 0:  # PU
                            pu_counts[valid_labels[i]] += 1
                        else:  # PUEA
                            puea_counts[valid_labels[i]] += 1
                    
                    print(f"    PU counts: {pu_counts}, PUEA counts: {puea_counts}")
                    
                    # Determine which cluster is the PUEA cluster
                    # The cluster with higher PUEA concentration is labeled as PUEA cluster
                    puea_concentration_0 = puea_counts[0] / (puea_counts[0] + pu_counts[0] + 0.0001)
                    puea_concentration_1 = puea_counts[1] / (puea_counts[1] + pu_counts[1] + 0.0001)
                    puea_cluster = 1 if puea_concentration_1 > puea_concentration_0 else 0
                    
                    print(f"    PUEA cluster: {puea_cluster} (concentrations: {puea_concentration_0:.2f}, {puea_concentration_1:.2f})")
                    
                    # Calculate detection rate and false detection rate
                    total_puea = puea_counts[0] + puea_counts[1]
                    total_pu = pu_counts[0] + pu_counts[1]
                    
                    puea_detection_rate = puea_counts[puea_cluster] / total_puea if total_puea > 0 else 0
                    false_detection_rate = pu_counts[puea_cluster] / total_pu if total_pu > 0 else 0
                    
                    # Calculate accuracy and ARI
                    # Map clusters to original labels (0=PU, 1=PUEA)
                    mapped_labels = np.zeros_like(valid_labels)
                    if puea_cluster == 1:
                        mapped_labels = valid_labels
                    else:
                        mapped_labels = 1 - valid_labels
                        
                    accuracy = np.mean(mapped_labels == valid_true) * 100
                    ari = adjusted_rand_score(valid_true, valid_labels)
                    
                    # Add to metrics dictionary
                    detection_metrics['case'].append(case)
                    detection_metrics['percentage'].append(percentage)
                    detection_metrics['algorithm'].append(alg_name)
                    detection_metrics['puea_detection_rate'].append(puea_detection_rate)
                    detection_metrics['false_detection_rate'].append(false_detection_rate)
                    detection_metrics['accuracy'].append(accuracy)
                    detection_metrics['ari'].append(ari)
                    
                    print(f"    {case}_{percentage}% - {alg_name}: DR={puea_detection_rate:.2f}, FDR={false_detection_rate:.2f}, Acc={accuracy:.1f}%, ARI={ari:.3f}")
                else:
                    print(f"    No clustering results found for {alg_key}")
        
        except Exception as e:
            print(f"Error processing metrics for {key}: {str(e)}")
            import traceback
            traceback.print_exc()

# Convert to DataFrame for easier analysis
metrics_df = pd.DataFrame(detection_metrics)

# Debug: Check what algorithms we have in the metrics
print("\nAlgorithms in metrics_df:", metrics_df['algorithm'].unique())
print("\nMetrics data shape:", metrics_df.shape)
print("\nMetrics data summary by algorithm:")
print(metrics_df.groupby('algorithm').size())

# Create combined performance metric plots
if not metrics_df.empty:
    # Create plots for each case showing detection rates by algorithm
    for case in cases:
        case_data = metrics_df[metrics_df['case'] == case]
        if case_data.empty:
            print(f"No data for case {case}, skipping plot")
            continue
            
        # Create figure with two subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        fig.suptitle(f'Case {case} - PUEA Detection Performance', fontsize=16)
        
        # Get unique algorithms for this case
        case_algorithms = sorted(case_data['algorithm'].unique())
        print(f"Case {case} has algorithms: {case_algorithms}")
        
        # Define distinct visual attributes for each algorithm
        algorithm_styles = {
            'DBSCAN': {
                'color': 'blue', 
                'marker': 'o', 
                'linestyle': '-',
                'linewidth': 2.5,
                'markersize': 8,
                'zorder': 3,
                'label': 'DBSCAN'
            },
            'K-means': {
                'color': 'red', 
                'marker': 's', 
                'linestyle': '--',
                'linewidth': 2,
                'markersize': 9,
                'zorder': 2,
                'label': 'K-means'
            },
            'Agglomerative': {
                'color': 'green', 
                'marker': '^', 
                'linestyle': '-.',
                'linewidth': 2.2,
                'markersize': 10,
                'zorder': 1,
                'label': 'Agglomerative'
            }
        }
        
        # Plot PUEA Detection Rate with enhanced visual distinction
        for alg in case_algorithms:
            alg_data = case_data[case_data['algorithm'] == alg].sort_values('percentage')
            if len(alg_data) == 0:
                print(f"  No data points for {alg}, skipping")
                continue
                
            print(f"Plotting {alg} with {len(alg_data)} points")
            style = algorithm_styles[alg]
            
            # Print data points being plotted
            print(f"  Detection rate data for {alg}: {list(zip(alg_data['percentage'], alg_data['puea_detection_rate']))}")
            
            # Plot with enhanced visibility
            line, = ax1.plot(alg_data['percentage'], alg_data['puea_detection_rate'], 
                    marker=style['marker'], 
                    linestyle=style['linestyle'], 
                    label=style['label'],
                    color=style['color'], 
                    linewidth=style['linewidth'], 
                    markersize=style['markersize'],
                    zorder=style['zorder'],
                    markeredgecolor='black',
                    markeredgewidth=0.8)
            
            # Add data point labels for clarity when lines overlap
            for x, y in zip(alg_data['percentage'], alg_data['puea_detection_rate']):
                ax1.annotate(f'{y:.2f}', 
                            xy=(x, y),
                            xytext=(0, 5),  # 5 points vertical offset
                            textcoords='offset points',
                            ha='center',
                            fontsize=8,
                            color=style['color'],
                            fontweight='bold')
        
        # Customize PUEA Detection Rate plot
        ax1.set_title('PUEA Detection Rate', fontsize=14)
        ax1.set_xlabel('PUEA Percentage (%)', fontsize=12)
        ax1.set_ylabel('Detection Rate', fontsize=12)
        ax1.set_xticks(percentages)
        ax1.set_ylim(0, 1.05)
        ax1.grid(True, alpha=0.3, linestyle='--')
        ax1.legend(loc='best', fontsize=10, framealpha=0.8, edgecolor='black')
        
        # Plot False Detection Rate with enhanced visual distinction
        for alg in case_algorithms:
            alg_data = case_data[case_data['algorithm'] == alg].sort_values('percentage')
            if len(alg_data) == 0:
                continue
                
            style = algorithm_styles[alg]
            
            # Print data points being plotted
            print(f"  False detection rate data for {alg}: {list(zip(alg_data['percentage'], alg_data['false_detection_rate']))}")
            
            # Plot with enhanced visibility
            line, = ax2.plot(alg_data['percentage'], alg_data['false_detection_rate'], 
                    marker=style['marker'], 
                    linestyle=style['linestyle'], 
                    label=style['label'],
                    color=style['color'], 
                    linewidth=style['linewidth'], 
                    markersize=style['markersize'],
                    zorder=style['zorder'],
                    markeredgecolor='black',
                    markeredgewidth=0.8)
                    
            # Add data point labels for clarity when lines overlap
            for x, y in zip(alg_data['percentage'], alg_data['false_detection_rate']):
                ax2.annotate(f'{y:.2f}', 
                            xy=(x, y),
                            xytext=(0, 5),  # 5 points vertical offset
                            textcoords='offset points',
                            ha='center',
                            fontsize=8,
                            color=style['color'],
                            fontweight='bold')
        
        # Customize False Detection Rate plot
        ax2.set_title('False Detection Rate', fontsize=14)
        ax2.set_xlabel('PUEA Percentage (%)', fontsize=12)
        ax2.set_ylabel('False Detection Rate', fontsize=12)
        ax2.set_xticks(percentages)
        ax2.set_ylim(0, 1.05)
        ax2.grid(True, alpha=0.3, linestyle='--')
        ax2.legend(loc='best', fontsize=10, framealpha=0.8, edgecolor='black')
        
        # Add case description title
        case_descriptions = {'A': 'Far Distance', 'B': 'Medium Distance', 'C': 'Close Distance'}
        if case in case_descriptions:
            fig.suptitle(f'Case {case} ({case_descriptions[case]}) - PUEA Detection Performance', fontsize=16)
        
        plt.tight_layout()
        plt.subplots_adjust(top=0.88)
        
        # Save figure
        plt.savefig(f"{perf_viz_dir}/case_{case}_detection_rates.png", dpi=300, bbox_inches='tight')
        plt.close()
    
    # Create a summary plot comparing algorithm performance across cases
    fig, axes = plt.subplots(len(cases), 2, figsize=(15, 4*len(cases)))
    
    for i, case in enumerate(cases):
        case_metrics = metrics_df[metrics_df['case'] == case]
        
        if case_metrics.empty:
            continue
            
        # Detection Rate plot
        ax1 = axes[i, 0]
        
        # Group by algorithm and percentage, calculate mean detection rate
        pivot_dr = case_metrics.pivot_table(
            index='percentage', 
            columns='algorithm', 
            values='puea_detection_rate',
            aggfunc='mean'
        )
        
        # Print pivot table for debugging
        print(f"\nPivot table for case {case} detection rates:")
        print(pivot_dr)
        
        # Plot detection rate lines with distinct styles
        colors = ['blue', 'red', 'green']
        markers = ['o', 's', '^']
        linestyles = ['-', '--', '-.']
        
        for j, alg in enumerate(pivot_dr.columns):
            ax1.plot(pivot_dr.index, pivot_dr[alg], 
                     marker=markers[j % len(markers)],
                     linestyle=linestyles[j % len(linestyles)],
                     color=colors[j % len(colors)],
                     linewidth=2,
                     markersize=8,
                     markeredgecolor='black',
                     markeredgewidth=0.8,
                     label=alg)
            
            # Add data labels
            for x, y in zip(pivot_dr.index, pivot_dr[alg]):
                ax1.annotate(f'{y:.2f}', 
                            xy=(x, y),
                            xytext=(0, 5),
                            textcoords='offset points',
                            ha='center',
                            fontsize=8,
                            color=colors[j % len(colors)],
                            fontweight='bold')
        
        ax1.set_title(f'Case {case} - PUEA Detection Rate', fontsize=13)
        ax1.set_xlabel('PUEA Percentage (%)', fontsize=11)
        ax1.set_ylabel('Detection Rate', fontsize=11)
        ax1.set_ylim(0, 1.05)
        ax1.set_xticks(percentages)
        ax1.grid(True, alpha=0.3, linestyle='--')
        ax1.legend(loc='best', fontsize=9)
        
        # False Detection Rate plot
        ax2 = axes[i, 1]
        
        # Group by algorithm and percentage, calculate mean false detection rate
        pivot_fdr = case_metrics.pivot_table(
            index='percentage', 
            columns='algorithm', 
            values='false_detection_rate',
            aggfunc='mean'
        )
        
        # Print pivot table for debugging
        print(f"\nPivot table for case {case} false detection rates:")
        print(pivot_fdr)
        
        # Plot false detection rate lines with distinct styles
        for j, alg in enumerate(pivot_fdr.columns):
            ax2.plot(pivot_fdr.index, pivot_fdr[alg], 
                     marker=markers[j % len(markers)],
                     linestyle=linestyles[j % len(linestyles)],
                     color=colors[j % len(colors)],
                     linewidth=2,
                     markersize=8,
                     markeredgecolor='black',
                     markeredgewidth=0.8,
                     label=alg)
            
            # Add data labels
            for x, y in zip(pivot_fdr.index, pivot_fdr[alg]):
                ax2.annotate(f'{y:.2f}', 
                            xy=(x, y),
                            xytext=(0, 5),
                            textcoords='offset points',
                            ha='center',
                            fontsize=8,
                            color=colors[j % len(colors)],
                            fontweight='bold')
        
        ax2.set_title(f'Case {case} - False Detection Rate', fontsize=13)
        ax2.set_xlabel('PUEA Percentage (%)', fontsize=11)
        ax2.set_ylabel('False Detection Rate', fontsize=11)
        ax2.set_ylim(0, 1.05)
        ax2.set_xticks(percentages)
        ax2.grid(True, alpha=0.3, linestyle='--')
        ax2.legend(loc='best', fontsize=9)
        
        # Add case description to y-axis label
        case_descriptions = {'A': 'Far Distance', 'B': 'Medium Distance', 'C': 'Close Distance'}
        if case in case_descriptions:
            ax1.set_ylabel(f"Detection Rate\nCase {case}: {case_descriptions[case]}", fontsize=11)
        
    plt.tight_layout()
    plt.savefig(f"{perf_viz_dir}/all_cases_detection_performance.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create a heatmap of detection performance by algorithm and case
    plt.figure(figsize=(12, 9))
    
    # Calculate average detection rate for each algorithm/case combination
    avg_metrics = metrics_df.groupby(['algorithm', 'case']).agg({
        'puea_detection_rate': 'mean',
        'false_detection_rate': 'mean',
        'accuracy': 'mean',
        'ari': 'mean'
    }).reset_index()
    
    # Print the aggregated metrics
    print("\nAggregated metrics by algorithm and case:")
    print(avg_metrics)
    
    # Create pivot table for heatmap
    pivot_metrics = avg_metrics.pivot_table(
        index='algorithm', 
        columns='case', 
        values='puea_detection_rate'
    )
    
    # Print the pivot table for the heatmap
    print("\nPivot table for heatmap:")
    print(pivot_metrics)
    
    # Plot heatmap (only if we have data)
    if not pivot_metrics.empty:
        cmap = sns.cm.rocket_r  # Using a perceptually uniform colormap
        sns.heatmap(pivot_metrics, annot=True, cmap=cmap, fmt='.2f', vmin=0, vmax=1, 
                   annot_kws={"size": 14, "weight": "bold"},
                   linewidths=0.5, linecolor='white', cbar_kws={"shrink": 0.8})
        plt.title('Average PUEA Detection Rate by Algorithm and Case', fontsize=16, pad=20)
        
        # Add case descriptions as second row of column labels
        case_descriptions = {'A': 'Far Distance', 'B': 'Medium Distance', 'C': 'Close Distance'}
        ax = plt.gca()
        for i, case in enumerate(pivot_metrics.columns):
            ax.text(i + 0.5, -0.15, case_descriptions.get(case, ''),
                   ha='center', va='center', fontsize=10, style='italic')
        
        plt.tight_layout()
        plt.savefig(f"{perf_viz_dir}/detection_rate_heatmap.png", dpi=300, bbox_inches='tight')
        plt.close()
    else:
        print("Warning: Empty pivot table for heatmap, skipping heatmap generation")
    
    # Save metrics to CSV for further analysis
    metrics_df.to_csv(f"{perf_viz_dir}/detection_performance_metrics.csv", index=False)
    print(f"\nDetection performance metrics and visualizations saved to {perf_viz_dir}/")
    
    # Print best performing algorithm for each case
    print("\n=== Best Performing Algorithm by Case ===")
    for case in cases:
        case_data = metrics_df[metrics_df['case'] == case]
        if not case_data.empty:
            # Group by algorithm and find the one with highest detection rate and lowest false detection rate
            avg_by_alg = case_data.groupby('algorithm').agg({
                'puea_detection_rate': 'mean',
                'false_detection_rate': 'mean',
                'accuracy': 'mean',
                'ari': 'mean'
            }).reset_index()
            
            # Score each algorithm (higher detection rate and lower false detection rate is better)
            avg_by_alg['score'] = avg_by_alg['puea_detection_rate'] - avg_by_alg['false_detection_rate']
            
            # Print scoring for all algorithms
            print(f"\nAlgorithm scoring for Case {case}:")
            for _, row in avg_by_alg.iterrows():
                print(f"  {row['algorithm']}: Score={row['score']:.3f}, DR={row['puea_detection_rate']:.3f}, FDR={row['false_detection_rate']:.3f}")
            
            # Get best algorithm
            best_alg = avg_by_alg.loc[avg_by_alg['score'].idxmax()]
            
            print(f"Case {case} - Best algorithm: {best_alg['algorithm']}")
            print(f"  Detection Rate: {best_alg['puea_detection_rate']:.3f}")
            print(f"  False Detection Rate: {best_alg['false_detection_rate']:.3f}")
            print(f"  Accuracy: {best_alg['accuracy']:.1f}%")
            print(f"  ARI: {best_alg['ari']:.3f}")
    
else:
    print("No detection metrics available. Make sure clustering results are generated correctly.")
    print("Available keys in clustering_results:", list(clustering_results.keys()))

In [ ]:
# Cell 5: Specialized Algorithms for PUEA Detection
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.metrics import adjusted_rand_score
import seaborn as sns
from collections import defaultdict

# Define the K-NN algorithm for PUEA detection
def knn_algorithm_puea_detection(distance_matrix, candidate_set, normal_set, R=5):
    """
    K-NN algorithm for PUEA detection
    
    Parameters:
        distance_matrix: Manhattan distance matrix
        candidate_set: Set of indices of suspected PUEA attackers
        normal_set: Set of indices of presumed legitimate users
        R: Number of nearest neighbors to consider
    
    Returns:
        outlier_set: Set of nodes classified as PUEA attackers
    """
    n = distance_matrix.shape[0]
    outlier_set = set()
    
    print(f"Running K-NN algorithm with R={R}")
    print(f"Initial candidate set size: {len(candidate_set)}")
    print(f"Initial normal set size: {len(normal_set)}")
    
    for i in candidate_set:
        # Sort distances to find nearest neighbors (exclude self)
        distances = [(j, distance_matrix[i, j]) for j in range(n) if i != j]
        distances.sort(key=lambda x: x[1])  # Sort by distance
        
        # Count neighbors in candidate vs normal set for the R nearest neighbors
        cCand = 0
        cNormal = 0
        
        for j in range(min(R, len(distances))):
            neighbor_idx = distances[j][0]
            if neighbor_idx in candidate_set:
                cCand += 1
            elif neighbor_idx in normal_set:
                cNormal += 1
        
        # Decision: if more neighbors are in normal set, it's an outlier (PUEA)
        if cCand <= cNormal:
            outlier_set.add(i)
    
    print(f"K-NN algorithm detected {len(outlier_set)} outliers")
    return outlier_set

# Modified means_algorithm_puea_detection function with explicit type conversion
def means_algorithm_puea_detection(distance_matrix, candidate_set, normal_set):
    """
    Means algorithm for PUEA detection
    
    Parameters:
        distance_matrix: Manhattan distance matrix
        candidate_set: Set of indices of suspected PUEA attackers
        normal_set: Set of indices of presumed legitimate users
    
    Returns:
        refined_candidate_set: Updated set of suspected PUEA attackers
        refined_normal_set: Updated set of normal users
    """
    n = distance_matrix.shape[0]
    
    # Initialize distance arrays - convert numpy integers to Python integers
    cand_dist = {int(i): 0 for i in candidate_set}
    norm_dist = {int(i): 0 for i in normal_set}
    
    print(f"Running Means algorithm")
    print(f"Initial candidate set size: {len(candidate_set)}")
    print(f"Initial normal set size: {len(normal_set)}")
    
    # Calculate distances for candidate set
    total_cand_dist = 0
    for i in candidate_set:
        i_int = int(i)  # Convert to Python integer
        for j in range(n):
            cand_dist[i_int] += distance_matrix[i, j]
        total_cand_dist += cand_dist[i_int]
    
    # Calculate distances for normal set
    total_norm_dist = 0
    for i in normal_set:
        i_int = int(i)  # Convert to Python integer
        for j in range(n):
            norm_dist[i_int] += distance_matrix[i, j]
        total_norm_dist += norm_dist[i_int]
    
    # Calculate means
    cand_dist_mean = total_cand_dist / len(candidate_set) if candidate_set else 0
    norm_dist_mean = total_norm_dist / len(normal_set) if normal_set else 0
    
    # Refine sets - move points that are closer to the normal mean than candidate mean
    refined_candidate_set = set(candidate_set)
    refined_normal_set = set(normal_set)
    
    reassigned_count = 0
    
    for i in list(candidate_set):  # Use list to avoid modification during iteration
        i_int = int(i)  # Convert to Python integer
        # If point is closer to normal set mean than candidate set mean
        if abs(cand_dist_mean - cand_dist[i_int]) > abs(norm_dist_mean - norm_dist[i_int]):
            refined_candidate_set.remove(i)
            refined_normal_set.add(i)
            # Update totals
            total_cand_dist -= cand_dist[i_int]
            total_norm_dist += cand_dist[i_int]
            reassigned_count += 1
            
            # Update means
            if refined_candidate_set:
                cand_dist_mean = total_cand_dist / len(refined_candidate_set)
            if refined_normal_set:
                norm_dist_mean = total_norm_dist / len(refined_normal_set)
    
    print(f"Means algorithm reassigned {reassigned_count} points")
    print(f"Final candidate set size: {len(refined_candidate_set)}")
    print(f"Final normal set size: {len(refined_normal_set)}")
    
    return refined_candidate_set, refined_normal_set

# Define the Hybrid K-NN algorithm for PUEA detection
def hybrid_knn_puea_detection(distance_matrix, candidate_set, k=5, outlier_fraction=0.2):
    """
    Hybrid K-NN algorithm for PUEA detection
    
    Parameters:
        distance_matrix: Manhattan distance matrix
        candidate_set: Set of indices of suspected PUEA attackers
        k: Number of nearest neighbors to consider
        outlier_fraction: Expected fraction of PUEA attackers in candidate set
    
    Returns:
        outlier_set: Set of nodes classified as PUEA attackers
    """
    n = distance_matrix.shape[0]
    outlier_set = set()
    
    # Convert candidate_set to list for indexing
    candidate_list = list(candidate_set)
    
    # Calculate target outlier count
    outlier_count = max(1, int(round(len(candidate_set) * outlier_fraction)))
    
    print(f"Running Hybrid K-NN algorithm with k={k}, outlier_fraction={outlier_fraction}")
    print(f"Target outlier count: {outlier_count}")
    print(f"Initial candidate set size: {len(candidate_set)}")
    
    # Calculate k-NN distances for each candidate
    knn_dist = {}
    
    for i in candidate_set:
        # Find distances to all other points
        distances = [(j, distance_matrix[i, j]) for j in range(n) if (i != j)]
        distances.sort(key=lambda x: x[1])  # Sort by distance
        
        # Get k nearest neighbors
        knn_dist[i] = sum(dist for _, dist in distances[:k])
    
    # Calculate mean KNN distance
    current_candidate_set = set(candidate_set)
    total_knn_dist = sum(knn_dist.values())
    
    # Iteratively detect outliers
    detected_outliers = 0
    iteration = 0
    max_iterations = 100  # Prevent infinite loop
    
    while detected_outliers < outlier_count and current_candidate_set and iteration < max_iterations:
        iteration += 1
        
        # Calculate mean k-NN distance
        knn_dist_mean = total_knn_dist / len(current_candidate_set) if current_candidate_set else 0
        
        # Find points with distance greater than mean
        outliers_this_iteration = []
        
        for i in list(current_candidate_set):
            if knn_dist[i] > knn_dist_mean:
                outliers_this_iteration.append(i)
        
        # If no outliers found in this iteration, break
        if not outliers_this_iteration:
            break
            
        # Add new outliers up to the target count
        for i in outliers_this_iteration:
            if detected_outliers < outlier_count:
                outlier_set.add(i)
                current_candidate_set.remove(i)
                total_knn_dist -= knn_dist[i]
                detected_outliers += 1
            else:
                break
    
    print(f"Hybrid K-NN algorithm detected {len(outlier_set)} outliers in {iteration} iterations")
    return outlier_set

# Function to apply all three detection algorithms to a specific case and percentage
def apply_detection_algorithms(case, percentage, distance_dir="distance_matrices"):
    """
    Apply all three detection algorithms to a specific case and percentage
    
    Parameters:
        case: Case identifier (A, B, or C)
        percentage: PUEA percentage in the dataset
        distance_dir: Directory containing distance matrices
    
    Returns:
        results: Dictionary containing detection results from all algorithms
    """
    # Load distance matrix
    distance_matrix_file = os.path.join(distance_dir, f"{case}_{percentage}_manhattan_dist.csv")
    if not os.path.exists(distance_matrix_file):
        print(f"Error: Distance matrix file {distance_matrix_file} not found.")
        return None
    
    distance_matrix = pd.read_csv(distance_matrix_file).values
    
    # Load original labels for validation
    original_file = f"{case}_case_{percentage}percent_matrix.csv"
    if not os.path.exists(original_file):
        print(f"Error: Original file {original_file} not found.")
        return None
    
    df = pd.read_csv(original_file)
    true_labels = df['Label'].values
    
    # Initial classification based on statistical features
    # For initial division, we can use K-means or another simple clustering
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=2, random_state=42).fit(df.iloc[:, :-1].values)
    initial_labels = kmeans.labels_
    
    # Create candidate and normal sets
    # Determine which cluster is more likely to be PUEA
    cluster_0_puea_count = np.sum(true_labels[initial_labels == 0] == 1)
    cluster_1_puea_count = np.sum(true_labels[initial_labels == 1] == 1)
    
    print(f"\nInitial clustering:")
    print(f"Cluster 0: {np.sum(initial_labels == 0)} points, {cluster_0_puea_count} are PUEA")
    print(f"Cluster 1: {np.sum(initial_labels == 1)} points, {cluster_1_puea_count} are PUEA")
    
    # The cluster with more PUEA labels becomes the candidate set
    if cluster_0_puea_count > cluster_1_puea_count:
        candidate_set = set(np.where(initial_labels == 0)[0])
        normal_set = set(np.where(initial_labels == 1)[0])
        print("Cluster 0 selected as candidate set")
    else:
        candidate_set = set(np.where(initial_labels == 1)[0])
        normal_set = set(np.where(initial_labels == 0)[0])
        print("Cluster 1 selected as candidate set")
    
    print(f"\nApplying algorithms for Case {case}, {percentage}% PUEA:")
    
    # Apply K-NN algorithm
    print("\n1. K-NN Algorithm:")
    knn_outliers = knn_algorithm_puea_detection(
        distance_matrix, candidate_set, normal_set, R=5)
    
    # Apply Means algorithm
    print("\n2. Means Algorithm:")
    refined_candidate, refined_normal = means_algorithm_puea_detection(
        distance_matrix, candidate_set, normal_set)
    
    # Apply Hybrid K-NN algorithm
    print("\n3. Hybrid K-NN Algorithm:")
    hybrid_outliers = hybrid_knn_puea_detection(
        distance_matrix, candidate_set, k=5, outlier_fraction=percentage/100)
    
    # Evaluate results against true labels
    print("\nEvaluation Results:")
    knn_metrics = evaluate_detection(knn_outliers, true_labels)
    means_metrics = evaluate_detection(refined_candidate, true_labels)
    hybrid_metrics = evaluate_detection(hybrid_outliers, true_labels)
    
    print(f"K-NN: DR={knn_metrics['puea_detection_rate']:.2f}, FDR={knn_metrics['false_detection_rate']:.2f}, Acc={knn_metrics['accuracy']:.2f}")
    print(f"Means: DR={means_metrics['puea_detection_rate']:.2f}, FDR={means_metrics['false_detection_rate']:.2f}, Acc={means_metrics['accuracy']:.2f}")
    print(f"Hybrid: DR={hybrid_metrics['puea_detection_rate']:.2f}, FDR={hybrid_metrics['false_detection_rate']:.2f}, Acc={hybrid_metrics['accuracy']:.2f}")
    
    results = {
        'knn': knn_metrics,
        'means': means_metrics,
        'hybrid': hybrid_metrics
    }
    
    return results

def evaluate_detection(detected_set, true_labels):
    """
    Evaluate detection performance
    
    Parameters:
        detected_set: Set of indices detected as PUEA
        true_labels: Array of true labels (0=PU, 1=PUEA)
    
    Returns:
        metrics: Dictionary containing detection metrics
    """
    # Create detected labels array
    detected_labels = np.zeros_like(true_labels)
    for idx in detected_set:
        detected_labels[idx] = 1
    
    # Calculate metrics
    tp = np.sum((detected_labels == 1) & (true_labels == 1))
    fp = np.sum((detected_labels == 1) & (true_labels == 0))
    tn = np.sum((detected_labels == 0) & (true_labels == 0))
    fn = np.sum((detected_labels == 0) & (true_labels == 1))
    
    puea_detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 0
    false_detection_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
    accuracy = (tp + tn) / len(true_labels)
    
    return {
        'puea_detection_rate': puea_detection_rate,
        'false_detection_rate': false_detection_rate,
        'accuracy': accuracy,
        'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn
    }

# Add this at the top of your Cell 5 code
def run_all_experiments():
    results = {}
    cases = ['A', 'B', 'C']
    percentages = [10, 20, 30, 40, 50]
    
    # Create directory for results if it doesn't exist
    results_dir = "specialized_algorithm_results"
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    
    # Process each case and percentage
    for case in cases:
        for percentage in percentages:
            key = f"{case}_{percentage}"
            print(f"\n{'='*50}")
            print(f"Processing {key}")
            print(f"{'='*50}")
            
            try:
                # Fix: Convert the numpy int64 indices to Python integers
                # This is the key fix for the "np.int64(0)" error
                result = apply_detection_algorithms(case, percentage)
                if result:
                    results[key] = result
            except Exception as e:
                print(f"Error processing {key}: {str(e)}")
                import traceback
                traceback.print_exc()  # Print the full traceback for better debugging
    
    # Save results to CSV
    results_df = []
    
    for key, result in results.items():
        case, percentage = key.split('_')
        
        for alg_name, metrics in result.items():
            results_df.append({
                'Case': case,
                'Percentage': percentage,
                'Algorithm': alg_name,
                'PUEA_Detection_Rate': metrics['puea_detection_rate'],
                'False_Detection_Rate': metrics['false_detection_rate'],
                'Accuracy': metrics['accuracy'],
                'TP': metrics['tp'],
                'FP': metrics['fp'],
                'TN': metrics['tn'],
                'FN': metrics['fn']
            })
    
    # Convert to DataFrame and save
    if results_df:
        df = pd.DataFrame(results_df)
        csv_file = f"{results_dir}/specialized_algorithms_results.csv"
        df.to_csv(csv_file, index=False)
        print(f"\nResults saved to {csv_file}")
        
        # Return DataFrame for visualization
        return df
    else:
        print("No results to save.")
        return None

def visualize_results(results_df):
    """
    Create visualizations for algorithm comparison
    
    Parameters:
        results_df: DataFrame with results from all algorithms
    """
    if results_df is None or results_df.empty:
        print("No results to visualize.")
        return
    
    # Create directory for visualizations if it doesn't exist
    viz_dir = "specialized_algorithm_visualizations"
    if not os.path.exists(viz_dir):
        os.makedirs(viz_dir)
    
    # Plot detection rates for each case
    cases = results_df['Case'].unique()
    
    # Create figure with appropriate layout
    plt.figure(figsize=(15, 4*len(cases)))
    
    # Create color map for algorithms
    colors = {'knn': 'blue', 'means': 'green', 'hybrid': 'red'}
    markers = {'knn': 'o', 'means': 's', 'hybrid': '^'}
    linestyles = {'knn': '-', 'means': '--', 'hybrid': '-.'}
    
    for i, case in enumerate(cases):
        # Filter data for this case
        case_data = results_df[results_df['Case'] == case]
        
        # Create detection rate and false detection rate subplots
        plt.subplot(len(cases), 2, 2*i+1)
        
        # Plot detection rate for each algorithm
        for alg in ['knn', 'means', 'hybrid']:
            alg_data = case_data[case_data['Algorithm'] == alg].sort_values('Percentage')
            if not alg_data.empty:
                plt.plot(alg_data['Percentage'], 
                         alg_data['PUEA_Detection_Rate'],
                         marker=markers[alg],
                         linestyle=linestyles[alg],
                         color=colors[alg],
                         label=f"{alg.capitalize()}")
                
                # Add data labels
                for x, y in zip(alg_data['Percentage'], alg_data['PUEA_Detection_Rate']):
                    plt.text(x, y+0.02, f'{y:.2f}', 
                             ha='center', va='bottom', fontsize=8, color=colors[alg])
        
        plt.title(f'Case {case} - PUEA Detection Rate')
        plt.xlabel('PUEA Percentage (%)')
        plt.ylabel('Detection Rate')
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.xticks(sorted(results_df['Percentage'].unique()))
        plt.ylim(0, 1.05)
        plt.legend()
        
        # False Detection Rate subplot
        plt.subplot(len(cases), 2, 2*i+2)
        
        # Plot false detection rate for each algorithm
        for alg in ['knn', 'means', 'hybrid']:
            alg_data = case_data[case_data['Algorithm'] == alg].sort_values('Percentage')
            if not alg_data.empty:
                plt.plot(alg_data['Percentage'], 
                         alg_data['False_Detection_Rate'],
                         marker=markers[alg],
                         linestyle=linestyles[alg],
                         color=colors[alg],
                         label=f"{alg.capitalize()}")
                
                # Add data labels
                for x, y in zip(alg_data['Percentage'], alg_data['False_Detection_Rate']):
                    plt.text(x, y+0.02, f'{y:.2f}', 
                             ha='center', va='bottom', fontsize=8, color=colors[alg])
        
        plt.title(f'Case {case} - False Detection Rate')
        plt.xlabel('PUEA Percentage (%)')
        plt.ylabel('False Detection Rate')
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.xticks(sorted(results_df['Percentage'].unique()))
        plt.ylim(0, 1.05)
        plt.legend()
    
    plt.tight_layout()
    plt.savefig(f"{viz_dir}/detection_rates_by_case.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create heatmaps for detection rates across all cases and algorithms
    plt.figure(figsize=(12, 8))
    
    # Aggregate data by algorithm and case
    pivot_dr = pd.pivot_table(
        results_df, 
        values='PUEA_Detection_Rate', 
        index=['Algorithm', 'Percentage'], 
        columns=['Case']
    ).round(2)
    
    # Create heatmap
    sns.heatmap(pivot_dr, annot=True, cmap='viridis', 
                fmt='.2f', linewidths=0.5, cbar_kws={'label': 'PUEA Detection Rate'})
    plt.title('PUEA Detection Rate by Algorithm, Case and Percentage')
    plt.tight_layout()
    plt.savefig(f"{viz_dir}/detection_rate_heatmap.png", dpi=300, bbox_inches='tight')
    plt.close()

    # Compare with existing clustering algorithms if results are available
    try:
        existing_results = f"performance_visualizations/detection_performance_metrics.csv"
        if os.path.exists(existing_results):
            existing_df = pd.read_csv(existing_results)
            # Normalize algorithm names for comparison
            existing_df['Algorithm'] = existing_df['algorithm'].str.lower()
            existing_df['PUEA_Detection_Rate'] = existing_df['puea_detection_rate']
            existing_df['False_Detection_Rate'] = existing_df['false_detection_rate']
            
            # Create comparison plots
            plt.figure(figsize=(16, 12))
            
            # Plot average detection rates across all cases
            plt.subplot(2, 1, 1)
            
            # Calculate average detection rates
            specialized_avg = results_df.groupby(['Algorithm', 'Percentage'])['PUEA_Detection_Rate'].mean().reset_index()
            existing_avg = existing_df.groupby(['Algorithm', 'percentage'])['PUEA_Detection_Rate'].mean().reset_index()
            existing_avg.rename(columns={'percentage': 'Percentage'}, inplace=True)
            
            # Plot specialized algorithms
            for alg in ['knn', 'means', 'hybrid']:
                alg_data = specialized_avg[specialized_avg['Algorithm'] == alg]
                plt.plot(alg_data['Percentage'], alg_data['PUEA_Detection_Rate'],
                         marker=markers[alg], linestyle=linestyles[alg],
                         color=colors[alg], linewidth=2.5,
                         label=f"Specialized {alg.capitalize()}")
            
            # Plot existing algorithms with different line styles
            existing_colors = {'dbscan': 'purple', 'k-means': 'orange', 'agglomerative': 'brown'}
            existing_markers = {'dbscan': 'x', 'k-means': '+', 'agglomerative': 'd'}
            
            for alg in existing_df['Algorithm'].unique():
                if alg.lower() in existing_colors:
                    alg_data = existing_avg[existing_avg['Algorithm'] == alg.lower()]
                    if not alg_data.empty:
                        plt.plot(alg_data['Percentage'], alg_data['PUEA_Detection_Rate'],
                                marker=existing_markers.get(alg.lower(), '*'),
                                linestyle=':', linewidth=1.5,
                                color=existing_colors.get(alg.lower(), 'gray'),
                                label=f"Classic {alg}")
            
            plt.title('Average PUEA Detection Rate: Specialized vs Classic Algorithms')
            plt.xlabel('PUEA Percentage (%)')
            plt.ylabel('Detection Rate')
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.xticks(sorted(results_df['Percentage'].unique()))
            plt.ylim(0, 1.05)
            plt.legend(loc='lower right')
            
            # Plot average false detection rates
            plt.subplot(2, 1, 2)
            
            # Calculate average false detection rates
            specialized_fdr = results_df.groupby(['Algorithm', 'Percentage'])['False_Detection_Rate'].mean().reset_index()
            existing_fdr = existing_df.groupby(['Algorithm', 'percentage'])['False_Detection_Rate'].mean().reset_index()
            existing_fdr.rename(columns={'percentage': 'Percentage'}, inplace=True)
            
            # Plot specialized algorithms
            for alg in ['knn', 'means', 'hybrid']:
                alg_data = specialized_fdr[specialized_fdr['Algorithm'] == alg]
                plt.plot(alg_data['Percentage'], alg_data['False_Detection_Rate'],
                         marker=markers[alg], linestyle=linestyles[alg],
                         color=colors[alg], linewidth=2.5,
                         label=f"Specialized {alg.capitalize()}")
            
            # Plot existing algorithms
            for alg in existing_df['Algorithm'].unique():
                if alg.lower() in existing_colors:
                    alg_data = existing_fdr[existing_fdr['Algorithm'] == alg.lower()]
                    if not alg_data.empty:
                        plt.plot(alg_data['Percentage'], alg_data['False_Detection_Rate'],
                                marker=existing_markers.get(alg.lower(), '*'),
                                linestyle=':', linewidth=1.5,
                                color=existing_colors.get(alg.lower(), 'gray'),
                                label=f"Classic {alg}")
            
            plt.title('Average False Detection Rate: Specialized vs Classic Algorithms')
            plt.xlabel('PUEA Percentage (%)')
            plt.ylabel('False Detection Rate')
            plt.grid(True, linestyle='--', alpha=0.6)
            plt.xticks(sorted(results_df['Percentage'].unique()))
            plt.ylim(0, 1.05)
            plt.legend(loc='upper right')
            
            plt.tight_layout()
            plt.savefig(f"{viz_dir}/specialized_vs_classic_comparison.png", dpi=300, bbox_inches='tight')
            plt.close()
            
            print(f"Comparison visualizations saved to {viz_dir}")
        else:
            print("No existing algorithm results found for comparison.")
            
    except Exception as e:
        print(f"Error creating comparison visualizations: {str(e)}")
    
    print(f"Visualizations saved to {viz_dir}")

# Main execution
print("Running specialized algorithms for PUEA detection...")
results_df = run_all_experiments()
if results_df is not None:
    visualize_results(results_df)

In [ ]:
# Cell 6: Enhanced Visualizations for PUEA Detection Results
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.gridspec import GridSpec

# Create directory for enhanced visualizations
viz_dir = "enhanced_visualizations"
if not os.path.exists(viz_dir):
    os.makedirs(viz_dir)

# Load results from specialized algorithms
specialized_results_file = "specialized_algorithm_results/specialized_algorithms_results.csv"
if os.path.exists(specialized_results_file):
    specialized_df = pd.read_csv(specialized_results_file)
    print(f"Loaded specialized algorithm results: {len(specialized_df)} entries")
else:
    specialized_df = None
    print("Specialized algorithm results not found, run Cell 5 first")

# Load results from clustering algorithms
clustering_results_file = "performance_visualizations/detection_performance_metrics.csv"
if os.path.exists(clustering_results_file):
    clustering_df = pd.read_csv(clustering_results_file)
    # Normalize column names to match with specialized algorithms
    clustering_df = clustering_df.rename(columns={
        'puea_detection_rate': 'PUEA_Detection_Rate',
        'false_detection_rate': 'False_Detection_Rate',
        'accuracy': 'Accuracy',
        'algorithm': 'Algorithm',
        'case': 'Case',
        'percentage': 'Percentage'
    })
    print(f"Loaded clustering algorithm results: {len(clustering_df)} entries")
else:
    clustering_df = None
    print("Clustering algorithm results not found, run Cell 4 first")

if specialized_df is not None or clustering_df is not None:
    # Create a combined DataFrame of all results
    combined_results = []
    
    if specialized_df is not None:
        for _, row in specialized_df.iterrows():
            combined_results.append({
                'Case': row['Case'],
                'Percentage': int(row['Percentage']),
                'Algorithm': f"Specialized_{row['Algorithm'].capitalize()}",
                'Algorithm_Type': 'Specialized',
                'Algorithm_Name': row['Algorithm'],
                'PUEA_Detection_Rate': row['PUEA_Detection_Rate'],
                'False_Detection_Rate': row['False_Detection_Rate'],
                'Accuracy': row['Accuracy'] * 100 if row['Accuracy'] <= 1 else row['Accuracy']
            })
    
    if clustering_df is not None:
        for _, row in clustering_df.iterrows():
            combined_results.append({
                'Case': row['Case'],
                'Percentage': int(row['Percentage']),
                'Algorithm': f"Clustering_{row['Algorithm']}",
                'Algorithm_Type': 'Clustering',
                'Algorithm_Name': row['Algorithm'],
                'PUEA_Detection_Rate': row['PUEA_Detection_Rate'],
                'False_Detection_Rate': row['False_Detection_Rate'],
                'Accuracy': row['Accuracy']
            })
    
    # Convert to DataFrame
    all_results_df = pd.DataFrame(combined_results)
    
    # ===============================================
    # 1. Create detection rate comparison by case
    # ===============================================
    cases = all_results_df['Case'].unique()
    percentages = sorted(all_results_df['Percentage'].unique())
    
    # Define color scheme
    colors = {
        'Specialized_Knn': '#1f77b4',      # Blue
        'Specialized_Means': '#2ca02c',    # Green
        'Specialized_Hybrid': '#d62728',   # Red
        'Clustering_DBSCAN': '#9467bd',    # Purple
        'Clustering_K-means': '#ff7f0e',   # Orange
        'Clustering_Agglomerative': '#8c564b'  # Brown
    }
    
    # Create multi-plot figure showing detection rates for each case
    plt.figure(figsize=(20, 15))
    
    for i, case in enumerate(cases):
        case_data = all_results_df[all_results_df['Case'] == case]
        
        # Plot PUEA Detection Rate
        plt.subplot(3, 2, 2*i+1)
        for alg in case_data['Algorithm'].unique():
            alg_data = case_data[case_data['Algorithm'] == alg].sort_values('Percentage')
            plt.plot(alg_data['Percentage'], alg_data['PUEA_Detection_Rate'], 
                    marker='o' if 'Specialized' in alg else '^',
                    linestyle='-' if 'Specialized' in alg else '--',
                    linewidth=2.5 if 'Specialized' in alg else 2.0,
                    color=colors.get(alg, 'gray'),
                    label=alg)
            
        plt.title(f'Case {case} - PUEA Detection Rate', fontsize=14)
        plt.xlabel('PUEA Percentage (%)', fontsize=12)
        plt.ylabel('Detection Rate', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.xticks(percentages)
        plt.ylim(0, 1.05)
        plt.legend(loc='lower right', fontsize=10)
        
        # Plot False Detection Rate
        plt.subplot(3, 2, 2*i+2)
        for alg in case_data['Algorithm'].unique():
            alg_data = case_data[case_data['Algorithm'] == alg].sort_values('Percentage')
            plt.plot(alg_data['Percentage'], alg_data['False_Detection_Rate'], 
                    marker='o' if 'Specialized' in alg else '^',
                    linestyle='-' if 'Specialized' in alg else '--',
                    linewidth=2.5 if 'Specialized' in alg else 2.0,
                    color=colors.get(alg, 'gray'),
                    label=alg)
            
        plt.title(f'Case {case} - False Detection Rate', fontsize=14)
        plt.xlabel('PUEA Percentage (%)', fontsize=12)
        plt.ylabel('False Detection Rate', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.xticks(percentages)
        plt.ylim(0, 1.05)
        plt.legend(loc='upper right', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(f"{viz_dir}/detection_comparison_by_case.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # ===============================================
    # 2. Create performance heatmaps
    # ===============================================
    # Function to create heatmap
    def create_performance_heatmap(metric, title, filename, cmap='RdYlGn'):
        plt.figure(figsize=(15, 10))
        
        # Create pivot table for heatmap
        pivot_data = pd.pivot_table(
            all_results_df, 
            values=metric,
            index=['Algorithm_Type', 'Algorithm_Name'],
            columns=['Case', 'Percentage']
        ).round(2)
        
        # Create custom colormap
        if metric == 'False_Detection_Rate':
            cmap = 'RdYlGn_r'  # Reversed colormap for FDR (lower is better)
            
        # Plot heatmap
        sns.heatmap(pivot_data, annot=True, cmap=cmap, fmt='.2f', 
                   linewidths=0.5, linecolor='white', 
                   cbar_kws={'label': metric.replace('_', ' ')})
        
        plt.title(title, fontsize=16, pad=20)
        plt.tight_layout()
        plt.savefig(f"{viz_dir}/{filename}", dpi=300, bbox_inches='tight')
        plt.close()
    
    # Create heatmaps for each metric
    create_performance_heatmap('PUEA_Detection_Rate', 
                             'PUEA Detection Rate Comparison Across Algorithms and Scenarios',
                             'puea_detection_rate_heatmap.png')
                             
    create_performance_heatmap('False_Detection_Rate', 
                             'False Detection Rate Comparison Across Algorithms and Scenarios',
                             'false_detection_rate_heatmap.png')
                             
    create_performance_heatmap('Accuracy', 
                             'Accuracy Comparison Across Algorithms and Scenarios',
                             'accuracy_heatmap.png')
    
    # ===============================================
    # 3. Create performance radar charts
    # ===============================================
    # Aggregate performance metrics by algorithm type
    avg_performance = all_results_df.groupby(['Algorithm_Type', 'Algorithm_Name']).agg({
        'PUEA_Detection_Rate': 'mean',
        'False_Detection_Rate': 'mean',
        'Accuracy': 'mean'
    }).reset_index()
    
    # Normalize values for radar chart (higher is better)
    avg_performance['Norm_Detection_Rate'] = avg_performance['PUEA_Detection_Rate']
    avg_performance['Norm_False_Rate'] = 1 - avg_performance['False_Detection_Rate']
    avg_performance['Norm_Accuracy'] = avg_performance['Accuracy'] / 100 if avg_performance['Accuracy'].max() > 1 else avg_performance['Accuracy']
    
    # Create radar chart
    categories = ['Detection Rate', 'Inverse False Rate', 'Accuracy']
    N = len(categories)
    
    # Create a figure with proper size
    fig = plt.figure(figsize=(12, 10))
    
    # Create custom color map for specialized vs clustering algorithms
    colors_radar = {
        'Specialized': ['#1f77b4', '#2ca02c', '#d62728'],
        'Clustering': ['#9467bd', '#ff7f0e', '#8c564b']
    }
    
    # Angle of each axis
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Close the loop
    
    # Initialize the subplot
    ax = plt.subplot(111, polar=True)
    
    # Draw one axis per variable and add labels
    plt.xticks(angles[:-1], categories, size=14)
    
    # Draw ylabels (0 to 1.0)
    ax.set_rlabel_position(0)
    plt.yticks([0.2, 0.4, 0.6, 0.8, 1.0], ["0.2", "0.4", "0.6", "0.8", "1.0"], size=12)
    plt.ylim(0, 1)
    
    # Plot each algorithm
    for alg_type in avg_performance['Algorithm_Type'].unique():
        type_data = avg_performance[avg_performance['Algorithm_Type'] == alg_type]
        type_colors = colors_radar[alg_type]
        
        for i, (_, row) in enumerate(type_data.iterrows()):
            # Calculate values for radar chart
            values = [row['Norm_Detection_Rate'], row['Norm_False_Rate'], row['Norm_Accuracy']]
            values += values[:1]  # Close the loop
            
            # Plot the values
            ax.plot(angles, values, linewidth=2, linestyle='-' if alg_type == 'Specialized' else '--',
                   label=f"{alg_type}_{row['Algorithm_Name']}", color=type_colors[i % len(type_colors)])
            ax.fill(angles, values, color=type_colors[i % len(type_colors)], alpha=0.1)
    
    # Add legend
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1), fontsize=12)
    
    plt.title('Algorithm Performance Comparison', size=20, pad=20)
    plt.tight_layout()
    plt.savefig(f"{viz_dir}/algorithm_radar_chart.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # ===============================================
    # 4. Create box plots for performance distribution
    # ===============================================
    plt.figure(figsize=(16, 12))
    
    # Create a grid for the boxplots
    gs = GridSpec(2, 2, height_ratios=[1, 1], width_ratios=[1, 1])
    
    # Detection Rate boxplot
    ax1 = plt.subplot(gs[0, 0])
    sns.boxplot(x='Algorithm_Type', y='PUEA_Detection_Rate', 
               hue='Algorithm_Name', data=all_results_df, ax=ax1)
    ax1.set_title('PUEA Detection Rate Distribution', fontsize=14)
    ax1.set_xlabel('')
    ax1.set_ylabel('Detection Rate', fontsize=12)
    ax1.legend(loc='upper right', fontsize=10)
    
    # False Detection Rate boxplot
    ax2 = plt.subplot(gs[0, 1])
    sns.boxplot(x='Algorithm_Type', y='False_Detection_Rate', 
               hue='Algorithm_Name', data=all_results_df, ax=ax2)
    ax2.set_title('False Detection Rate Distribution', fontsize=14)
    ax2.set_xlabel('')
    ax2.set_ylabel('False Detection Rate', fontsize=12)
    ax2.legend(loc='upper right', fontsize=10)
    
    # Case comparison boxplot
    ax3 = plt.subplot(gs[1, 0:])
    sns.boxplot(x='Case', y='PUEA_Detection_Rate', 
               hue='Algorithm_Type', data=all_results_df, ax=ax3)
    ax3.set_title('Detection Performance by Network Scenario', fontsize=14)
    ax3.set_xlabel('Network Scenario', fontsize=12)
    ax3.set_ylabel('PUEA Detection Rate', fontsize=12)
    ax3.legend(loc='lower right', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(f"{viz_dir}/performance_distributions.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # ===============================================
    # 5. Create a comprehensive summary chart
    # ===============================================
    # Calculate average performance metrics across all scenarios
    overall_perf = all_results_df.groupby(['Algorithm', 'Algorithm_Type', 'Algorithm_Name']).agg({
        'PUEA_Detection_Rate': 'mean',
        'False_Detection_Rate': 'mean',
        'Accuracy': 'mean'
    }).reset_index()
    
    # Sort by detection rate
    overall_perf = overall_perf.sort_values('PUEA_Detection_Rate', ascending=False)
    
    # Create a bar chart comparing all algorithms
    plt.figure(figsize=(14, 10))
    
    # Set up bar positions
    algorithms = overall_perf['Algorithm'].unique()
    x = np.arange(len(algorithms))
    width = 0.25
    
    # Create bars for each metric
    plt.bar(x - width, overall_perf['PUEA_Detection_Rate'], width, 
            label='Detection Rate', color='green', alpha=0.7)
    plt.bar(x, overall_perf['Accuracy']/100, width, 
            label='Accuracy', color='blue', alpha=0.7)
    plt.bar(x + width, overall_perf['False_Detection_Rate'], width, 
            label='False Detection Rate', color='red', alpha=0.7)
    
    # Customize plot
    plt.xlabel('Algorithm', fontsize=14)
    plt.ylabel('Performance Metric', fontsize=14)
    plt.title('Overall Algorithm Performance Comparison', fontsize=16)
    plt.xticks(x, algorithms, rotation=45, ha='right')
    plt.ylim(0, 1.05)
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, alg in enumerate(algorithms):
        row = overall_perf[overall_perf['Algorithm'] == alg].iloc[0]
        plt.text(i - width, row['PUEA_Detection_Rate'] + 0.02, f"{row['PUEA_Detection_Rate']:.2f}", 
                ha='center', va='bottom', fontsize=9, color='darkgreen')
        plt.text(i, row['Accuracy']/100 + 0.02, f"{row['Accuracy']:.1f}%", 
                ha='center', va='bottom', fontsize=9, color='darkblue')
        plt.text(i + width, row['False_Detection_Rate'] + 0.02, f"{row['False_Detection_Rate']:.2f}", 
                ha='center', va='bottom', fontsize=9, color='darkred')
    
    plt.tight_layout()
    plt.savefig(f"{viz_dir}/overall_algorithm_comparison.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Enhanced visualizations saved to {viz_dir}/ directory")
else:
    print("Cannot create visualizations: No algorithm results available")